In [2]:
import pandas as pd
df = pd.read_csv('Prot2text_UniProt.csv')

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 256690 entries, 0 to 256689
Data columns (total 34 columns):
 #   Column                              Non-Null Count   Dtype  
---  ------                              --------------   -----  
 0   Entry                               256690 non-null  object 
 1   name                                256690 non-null  object 
 2   Full Name                           255980 non-null  object 
 3   taxon                               256392 non-null  object 
 4   sequence                            256690 non-null  object 
 5   function                            256690 non-null  object 
 6   AlphaFoldDB                         256690 non-null  object 
 7   Protein names                       256681 non-null  object 
 8   Gene Names                          247647 non-null  object 
 9   Organism                            256681 non-null  object 
 10  Length                              256681 non-null  float64
 11  Active site               

In [10]:
import pandas as pd
import IPython.display as disp

# Build coverage table
def show_coverage(df):
    coverage = []
    total_rows = len(df)

    for i, col in enumerate(df.columns):
        non_null = df[col].notna().sum()
        unique_count = df[col].nunique(dropna=True)
        coverage.append({
            "Col #": i,
            "Column": col,
            "Non-null count": non_null,
            "Coverage %": round((non_null / total_rows) * 100, 2),
            "Unique count": unique_count
        })

    coverage_df = pd.DataFrame(coverage)

    # Sort by coverage % descending (optional)
    coverage_df = coverage_df.sort_values("Coverage %", ascending=False)

    # Show as a nice table

    disp.display(coverage_df)


In [5]:
# Print 3 unique values from each column 
i = 1
for col in df.columns:
    unique_vals = df[col].dropna().unique()[:3]
    print(f"# {i}---{col}: {unique_vals}---")
    i += 1

# 1---Entry: ['B7LNJ1' 'Q6LG09' 'Q01345']---
# 2---name: ['LPLT_ESCF3' 'OTC_PHOPR' 'NHEB_ONCMY']---
# 3---sequence: ['MSESVHTNTSLWSKGMKAVIVAQFLSAFGDNALLFATLALLKAQFYPEWSQPVLQMVFVGAYILFAPFVGQVADSFAKGRVMMFANGLKLLGAASICFGFNPFVGYTLVGIGAAAYSPAKYGILGELTTGDKLVKANGLMEASTIAAILLGSVAGGVLADLHVLVALAACALAYAGAVAANIYIPKLAAARPGQSWNVLKMTCSFKSACTSLWQNGETRFSLVGTSLFWGAGVTLRFLLVLWVPVALGITDNATPTYLNAMVAIGIVLGAGAAAKLVTLETVSRCMPAGILIGVVVLFFSLQHELLPAYALLMLIGVLGGFFVVPLNALLQERGKKSVGAGNAIAVQNLGENSAMLLMLGIYSLAVLVGIPVVPIGIGFGTLFALAITALWIWQRRH'
 'MSFNLRNRNFLKLLDFTGKEIEHLIALAQDLKHAKYAGTEQQKLKGKNIALIFEKTSTRTRCAFEVAAHDQGAHVTYIGGGSQMGHKESTKDTARVLGRFYDGIEYRGFGQDVVETLGEHAGVPVWNGLTDEWHPTQIIADWMTMLEHGNGKRLNQMKLAYMGDAKNNMGNSLMVGAAKVGMEIRLVGPKAYWPDPALVAECNELCKISGGKIVITDNVQEGVDSVDFIYGDVWVSMGEPEELWATRINDLAPYQVNMSVITATQNPAVKYMHCLPAFHNDETRVGKKIEEKFNMKGLEVTEDVFESSYTICFDEAENRMHSIKAIMVATLGD'
 'MPAFSCAFPGCRRDLLVIVLVVFVGIGLPIEASAPAYQSHGTEGSHLTNITNTKKAFPVLAVNYEHVRKPFEIALWILLALLMKLGFHLIPRLSAVVPESCLLIVVGLLVGGLIKVIGEEPPVLDSQLFFLCLLPPIILDAGYF

In [7]:
import itertools
import pandas as pd

# Always-take columns (100% coverage)
core_cols = ["Entry", "name", "sequence", "function", "Organism", "Length"]

# Candidate columns to test
candidates = [
    "Gene Names",
    "Gene Ontology (molecular function)",
    "Gene Ontology (biological process)",
    "Gene Ontology (cellular component)",
    "Subcellular location [CC]",
    "EC_final",
    "Catalytic activity",
    "reactions",
    "Rhea ID",
    "ph_optimum",
    "Binding site",
    "temperature_optimum",
    "km_values",
    "kcat_values"
]

results = []

# Try subsets of size 5 to all 12
for k in range(5, len(candidates)+1):
    best_combo = None
    best_cov = 0
    for combo in itertools.combinations(candidates, k):
        mask = df[list(combo)].notna().any(axis=1)
        coverage = mask.mean() * 100
        if coverage > best_cov:
            best_cov = coverage
            best_combo = combo
    results.append((k, best_cov, best_combo))

# Print results
for k, cov, combo in results:
    print(f"Top {k} cols: coverage={cov:.2f}%")
    print("  ", combo)


Top 5 cols: coverage=99.96%
   ('Gene Names', 'Gene Ontology (molecular function)', 'Gene Ontology (biological process)', 'Gene Ontology (cellular component)', 'Subcellular location [CC]')
Top 6 cols: coverage=99.96%
   ('Gene Names', 'Gene Ontology (molecular function)', 'Gene Ontology (biological process)', 'Gene Ontology (cellular component)', 'Subcellular location [CC]', 'EC_final')
Top 7 cols: coverage=99.96%
   ('Gene Names', 'Gene Ontology (molecular function)', 'Gene Ontology (biological process)', 'Gene Ontology (cellular component)', 'Subcellular location [CC]', 'EC_final', 'Catalytic activity')
Top 8 cols: coverage=99.96%
   ('Gene Names', 'Gene Ontology (molecular function)', 'Gene Ontology (biological process)', 'Gene Ontology (cellular component)', 'Subcellular location [CC]', 'EC_final', 'Catalytic activity', 'reactions')
Top 9 cols: coverage=99.96%
   ('Gene Names', 'Gene Ontology (molecular function)', 'Gene Ontology (biological process)', 'Gene Ontology (cellular comp

In [ ]:

"""
Needs Checking -> 
20	20	Domain [FT]	63134	24.60	36131 x -> ENrichmennt interpro, Pfam, SCope
19	19	Domain [CC]	23271	9.07	7099 -> Drop

Keep 1 ->
1	1	name	256690	100.00	256690 -> Keep 
4	4	Gene Names	247647	96.48	2307451 -> Keep 

Deop ->
30	30	ph_optimum	117435	45.75	2967
31	31	temperature_optimum	115511	45.00	2133
28	28	km_values	115494	44.99	3510
29	29	kcat_values	102786	40.04	2754
"""



'\n- Core Identity -> Entry, sequence, Organism, Length\n\n- Ontology/function -> Gene Ontology (molecular function), Gene Ontology (biological process), Gene Ontology (cellular component), EC_final, Catalytic activity, Rhea ID\n\n- Mechanistic features -> "InterPro_domains", Active_site (Active_site, Binding site, Site), Cofactor\n\n- Context -> Subcellular_location, Pathway (Pathway, KEGG_Pathways, UniPathway, Reactome)\n\n- External enrichments: PPI (STRING with score), Drug_target (DrugBank/TTD)\n\n- Provenance: evidence codes, source DBs, references\n\n-> Answer: Function\n\nLookFor -> PPI (STRING with score), Drug_target (DrugBank/TTD)\nLikelyInfered -> Topological domain, Motif, DNA binding\n\nNeeds Checking -> \n20\t20\tDomain [FT]\t63134\t24.60\t36131 x\n19\t19\tDomain [CC]\t23271\t9.07\t7099\n\nKeep 1 ->\n1\t1\tname\t256690\t100.00\t256690 \n4\t4\tGene Names\t247647\t96.48\t2307451\n\nDeop ->\n\n'

In [12]:
df = df.drop(columns=["taxon", "AlphaFoldDB", "Full Name", "Domain [CC]"])

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 256690 entries, 0 to 256689
Data columns (total 30 columns):
 #   Column                              Non-Null Count   Dtype  
---  ------                              --------------   -----  
 0   Entry                               256690 non-null  object 
 1   name                                256690 non-null  object 
 2   sequence                            256690 non-null  object 
 3   function                            256690 non-null  object 
 4   Protein names                       256681 non-null  object 
 5   Gene Names                          247647 non-null  object 
 6   Organism                            256681 non-null  object 
 7   Length                              256681 non-null  float64
 8   Active site                         55837 non-null   object 
 9   Catalytic activity                  132908 non-null  object 
 10  Binding site                        115900 non-null  object 
 11  Cofactor                  

In [14]:
df.to_csv("Prot2text_UniProt_final.csv", index=False)

In [ ]:
"""
- Core Identity -> @Entry (100%), @sequence (100%), Organism (100%), Length (100%), name (100%), Domain [FT] (24.60%)

- Ontology/function -> Gene Ontology (molecular function) (91.52%), Gene Ontology (biological process) (86.50%), 
   Gene Ontology (cellular component) (83.88%), EC_final (55.31%), Catalytic activity (51.78%), 

- Mechanistic features -> InterPro_domains (NA), Active site (21.75%) / Binding site (45.15%) / Site (7.12%), 
   cofactors (32.74%) / Cofactor (25.42%)

- Context -> Subcellular location [CC] (65.36%), Pathway (26.76%) / KEGG_Pathways (NA) / UniPathway (26.02%) / Reactome (7.20%)

- Likely Infered -> Topological domain (5.21%), Motif (4.68%), DNA binding (1.90%)

-> Answer -> function (100%)

-> ID COLS -> Rhea ID (46.83%) / SABIO-RK (1.30%) / Reactome (7.20%) / KEGG_Pathways (26.76%) / UniPathway (26.02%) 

"""

# - Core Identity -> Entry (100%), sequence (100%), Organism (100%), Length (100%), name (100%), Domain [FT] (24.60%)

In [20]:
df['Protein names'].unique()  # show top 20

array(['Lysophospholipid transporter LplT',
       'Ornithine carbamoyltransferase (OTCase) (EC 2.1.3.3)',
       'Na(+)/H(+) exchanger beta (Beta-NHE) (Na(+)/H(+) antiporter)',
       ..., 'Protein PET20, mitochondrial (Petite colonies protein 20)',
       'Fluoroquinolones export ATP-binding protein Rv2688c (EC 7.6.2.-)',
       'Homeotic protein spalt-major'], dtype=object)

In [22]:
df['name'].unique()  # show top 20

array(['LPLT_ESCF3', 'OTC_PHOPR', 'NHEB_ONCMY', ..., 'BSHC_CARHZ',
       'TERB2_DANRE', 'SALM_DROME'], dtype=object)

In [23]:
df["Gene Names"].unique()  # show top 20

array(['lplT EFER_2769', 'argF PBPRB1924', nan, ..., 'bshC CHY_1902',
       'terb2 zgc:111992', 'salm sal CG6464'], dtype=object)

In [32]:
import re

def extract_scientific_name(org):
    if pd.isna(org):
        return org
    # take text before first '('
    cleaned = org.split("(")[0].strip()
    # special case: if it ends with "sp", add back the dot
    if re.search(r"\bsp$", cleaned):
        cleaned += "."
    # remove accidental trailing slashes or spaces
    cleaned = re.sub(r"[\s/]+$", "", cleaned)
    return cleaned

# test again on 100 random rows
df_test = df.sample(100, random_state=42).copy()
df_test["Organism_clean"] = df_test["Organism"].apply(extract_scientific_name)

df_test[["Organism", "Organism_clean"]].head(10)

,Organism,Organism_clean
78438,Prochlorococcus marinus subsp. pastoris (strai...,Prochlorococcus marinus subsp. pastoris
137143,Homo sapiens (Human),Homo sapiens
250151,Leptospira interrogans serogroup Icterohaemorr...,Leptospira interrogans serogroup Icterohaemorr...
114992,Debaryomyces hansenii (strain ATCC 36239 / CBS...,Debaryomyces hansenii
181736,Marchantia polymorpha (Common liverwort) (Marc...,Marchantia polymorpha
109480,Bartonella bacilliformis (strain ATCC 35685 / ...,Bartonella bacilliformis
129102,Dictyostelium discoideum (Social amoeba),Dictyostelium discoideum
98468,Wolbachia sp. subsp. Brugia malayi (strain TRS),Wolbachia sp. subsp. Brugia malayi
101893,Geotalea uraniireducens (strain Rf4) (Geobacte...,Geotalea uraniireducens
105045,Bartonella quintana (strain Toulouse) (Rochali...,Bartonella quintana


In [33]:
# apply cleaning to full dataset
df["Organism"] = df["Organism"].apply(extract_scientific_name)


In [34]:
df['Organism'].unique()  # show top 20

array(['Escherichia fergusonii', 'Photobacterium profundum',
       'Oncorhynchus mykiss', ..., 'Sulfitobacter sp.',
       'Lobophytum crassum', 'Streptomyces pratensis'], dtype=object)

In [35]:
df = df.drop(columns=["Protein names", 'Gene Names'])

In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 256690 entries, 0 to 256689
Data columns (total 28 columns):
 #   Column                              Non-Null Count   Dtype  
---  ------                              --------------   -----  
 0   Entry                               256690 non-null  object 
 1   name                                256690 non-null  object 
 2   sequence                            256690 non-null  object 
 3   function                            256690 non-null  object 
 4   Organism                            256681 non-null  object 
 5   Length                              256681 non-null  float64
 6   Active site                         55837 non-null   object 
 7   Catalytic activity                  132908 non-null  object 
 8   Binding site                        115900 non-null  object 
 9   Cofactor                            65251 non-null   object 
 10  DNA binding                         4886 non-null    object 
 11  EC number                 

In [37]:
df['Domain [FT]'].unique()

array([nan, 'DOMAIN 30..157; /note="SCP"',
       'DOMAIN 80..175; /note="Toprim"; /evidence="ECO:0000255|HAMAP-Rule:MF_00017"',
       ...,
       'DOMAIN 194..428; /note="Grh/CP2 DB"; /evidence="ECO:0000255|PROSITE-ProRule:PRU01313"',
       'DOMAIN 43..489; /note="Trm1 methyltransferase"; /evidence="ECO:0000255|PROSITE-ProRule:PRU00958"',
       'DOMAIN 18..246; /note="ABC transporter"; /evidence="ECO:0000255|PROSITE-ProRule:PRU00434"'],
      dtype=object)

# Ontology/function -> Gene Ontology (molecular function) (91.52%), Gene Ontology (biological process) (86.50%), Gene Ontology (cellular component) (83.88%), EC number (55.31%), Catalytic activity (51.78%), Rhea ID (46.83%) / SABIO-RK (1.30%)

In [38]:
df['Gene Ontology (molecular function)'].unique()


array(['lysophospholipid:sodium symporter activity [GO:0051978]',
       'amino acid binding [GO:0016597]; ornithine carbamoyltransferase activity [GO:0004585]',
       'potassium:proton antiporter activity [GO:0015386]; sodium:proton antiporter activity [GO:0015385]',
       ...,
       'DNA binding [GO:0003677]; guanyl-nucleotide exchange factor activity [GO:0005085]; small GTPase binding [GO:0031267]; zinc ion binding [GO:0008270]',
       'DNA-binding transcription activator activity, RNA polymerase II-specific [GO:0001228]; RNA polymerase II cis-regulatory region sequence-specific DNA binding [GO:0000978]; sequence-specific DNA binding [GO:0043565]; sequence-specific double-stranded DNA binding [GO:1990837]',
       'tRNA (guanine(26)-N2)-dimethyltransferase activity [GO:0160104]; tRNA (guanine(26)-N2/guanine(27)-N2)-dimethyltransferase activity [GO:0160103]; tRNA binding [GO:0000049]'],
      dtype=object)

In [39]:
df['Gene Ontology (biological process)'].unique()

array([nan,
       'arginine biosynthetic process via ornithine [GO:0042450]; citrulline biosynthetic process [GO:0019240]; L-arginine biosynthetic process [GO:0006526]',
       'regulation of intracellular pH [GO:0051453]; sodium ion import across plasma membrane [GO:0098719]',
       ...,
       'compound eye development [GO:0048749]; compound eye morphogenesis [GO:0001745]; cuticle pattern formation [GO:0035017]; dorsal/ventral pattern formation, imaginal disc [GO:0007450]; female germ-line stem cell population maintenance [GO:0036099]; follicle cell of egg chamber development [GO:0030707]; germarium-derived egg chamber formation [GO:0007293]; imaginal disc-derived leg morphogenesis [GO:0007480]; imaginal disc-derived leg segmentation [GO:0036011]; imaginal disc-derived wing margin morphogenesis [GO:0008587]; imaginal disc-derived wing morphogenesis [GO:0007476]; negative regulation of Notch signaling pathway [GO:0045746]; positive regulation of Notch signaling pathway [GO:0045747];

In [40]:
df['Gene Ontology (cellular component)'].unique()

array(['plasma membrane [GO:0005886]', 'cytoplasm [GO:0005737]',
       'basolateral plasma membrane [GO:0016323]', ...,
       'cytosol [GO:0005829]; early endosome [GO:0005769]; endocytic vesicle [GO:0030139]; recycling endosome [GO:0055037]',
       'cytoplasm [GO:0005737]; Golgi medial cisterna [GO:0005797]; Golgi membrane [GO:0000139]; Golgi stack [GO:0005795]',
       'mitochondrion [GO:0005739]; nuclear envelope [GO:0005635]; nuclear inner membrane [GO:0005637]; nucleus [GO:0005634]'],
      dtype=object)

In [41]:
df['EC number'].unique()

array([nan, '2.1.3.3', '3.4.22.-', ..., '5.1.1.12', '2.1.1.333',
       '4.1.2.46'], dtype=object)

In [42]:
df['Catalytic activity'].unique()

array([nan,
       'CATALYTIC ACTIVITY: Reaction=carbamoyl phosphate + L-ornithine = L-citrulline + phosphate + H(+); Xref=Rhea:RHEA:19513, ChEBI:CHEBI:15378, ChEBI:CHEBI:43474, ChEBI:CHEBI:46911, ChEBI:CHEBI:57743, ChEBI:CHEBI:58228; EC=2.1.3.3; Evidence={ECO:0000255|HAMAP-Rule:MF_01109};',
       'CATALYTIC ACTIVITY: Reaction=tRNA(Trp) + L-tryptophan + ATP = L-tryptophyl-tRNA(Trp) + AMP + diphosphate + H(+); Xref=Rhea:RHEA:24080, Rhea:RHEA-COMP:9671, Rhea:RHEA-COMP:9705, ChEBI:CHEBI:15378, ChEBI:CHEBI:30616, ChEBI:CHEBI:33019, ChEBI:CHEBI:57912, ChEBI:CHEBI:78442, ChEBI:CHEBI:78535, ChEBI:CHEBI:456215; EC=6.1.1.2; Evidence={ECO:0000255|HAMAP-Rule:MF_00140};',
       ...,
       'CATALYTIC ACTIVITY: Reaction=3-O-(alpha-L-fucosyl)-L-threonyl-[EGF-like domain protein] + UDP-N-acetyl-alpha-D-glucosamine = 3-O-(N-acetyl-beta-D-glucosaminyl-(1->3)-alpha-L-fucosyl)-L-threonyl-[EGF-like domain protein] + UDP + H(+); Xref=Rhea:RHEA:70531, Rhea:RHEA-COMP:17922, Rhea:RHEA-COMP:17923, ChEBI:CHEB

# Mechanistic features -> Active site (21.75%) / Binding site (45.15%) / Site (7.12%), cofactors (32.74%) / Cofactor (25.42%)


In [43]:
df["Active site"].unique()

array([nan,
       'ACT_SITE 262; /evidence="ECO:0000255|PROSITE-ProRule:PRU10088"; ACT_SITE 397; /evidence="ECO:0000255|PROSITE-ProRule:PRU10088"; ACT_SITE 415; /evidence="ECO:0000255|PROSITE-ProRule:PRU10088"',
       'ACT_SITE 154; /note="Nucleophile"; /evidence="ECO:0000269|PubMed:15081808"; ACT_SITE 215; /note="Charge relay system"; /evidence="ECO:0000305|PubMed:15103133"; ACT_SITE 268; /note="Charge relay system"; /evidence="ECO:0000305|PubMed:15103133"',
       ...,
       'ACT_SITE 246; /note="Nucleophile"; /evidence="ECO:0000305|PubMed:16797589"',
       'ACT_SITE 259; /note="Charge relay system"; /evidence="ECO:0000250"; ACT_SITE 282; /note="Charge relay system"; /evidence="ECO:0000250"; ACT_SITE 333; /note="Charge relay system"; /evidence="ECO:0000250"',
       'ACT_SITE 327; /evidence="ECO:0000250"'], dtype=object)

In [44]:
df["Binding site"].unique()

array([nan,
       'BINDING 57..60; /ligand="carbamoyl phosphate"; /ligand_id="ChEBI:CHEBI:58228"; /evidence="ECO:0000255|HAMAP-Rule:MF_01109"; BINDING 83; /ligand="carbamoyl phosphate"; /ligand_id="ChEBI:CHEBI:58228"; /evidence="ECO:0000255|HAMAP-Rule:MF_01109"; BINDING 107; /ligand="carbamoyl phosphate"; /ligand_id="ChEBI:CHEBI:58228"; /evidence="ECO:0000255|HAMAP-Rule:MF_01109"; BINDING 134..137; /ligand="carbamoyl phosphate"; /ligand_id="ChEBI:CHEBI:58228"; /evidence="ECO:0000255|HAMAP-Rule:MF_01109"; BINDING 168; /ligand="L-ornithine"; /ligand_id="ChEBI:CHEBI:46911"; /evidence="ECO:0000255|HAMAP-Rule:MF_01109"; BINDING 232; /ligand="L-ornithine"; /ligand_id="ChEBI:CHEBI:46911"; /evidence="ECO:0000255|HAMAP-Rule:MF_01109"; BINDING 236..237; /ligand="L-ornithine"; /ligand_id="ChEBI:CHEBI:46911"; /evidence="ECO:0000255|HAMAP-Rule:MF_01109"; BINDING 274..275; /ligand="carbamoyl phosphate"; /ligand_id="ChEBI:CHEBI:58228"; /evidence="ECO:0000255|HAMAP-Rule:MF_01109"; BINDING 319; /ligan

In [45]:
df['Site'].unique()

array([nan,
       'SITE 228; /note="Important for catalytic activity"; /evidence="ECO:0000255|HAMAP-Rule:MF_01962"',
       'SITE 259..260; /note="Cleavage (non-hydrolytic); by autocatalysis"; /evidence="ECO:0000255|HAMAP-Rule:MF_00663"',
       ...,
       'SITE 112; /note="Interaction with substrate tRNA"; /evidence="ECO:0000255|HAMAP-Rule:MF_00185"',
       'SITE 737; /note="Arginine finger; crucial for GTP hydrolysis by stabilizing the transition state"; /evidence="ECO:0000255|PROSITE-ProRule:PRU00172"',
       'SITE 95; /note="Important for catalytic activity"; /evidence="ECO:0000269|PubMed:17251184"'],
      dtype=object)

In [46]:
df = df.drop(columns=["Site"])

In [47]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 256690 entries, 0 to 256689
Data columns (total 27 columns):
 #   Column                              Non-Null Count   Dtype  
---  ------                              --------------   -----  
 0   Entry                               256690 non-null  object 
 1   name                                256690 non-null  object 
 2   sequence                            256690 non-null  object 
 3   function                            256690 non-null  object 
 4   Organism                            256681 non-null  object 
 5   Length                              256681 non-null  float64
 6   Active site                         55837 non-null   object 
 7   Catalytic activity                  132908 non-null  object 
 8   Binding site                        115900 non-null  object 
 9   Cofactor                            65251 non-null   object 
 10  DNA binding                         4886 non-null    object 
 11  EC number                 

In [49]:
df['Cofactor'].unique()

array([nan,
       'COFACTOR: Name=Mn(2+); Xref=ChEBI:CHEBI:29035; Evidence={ECO:0000255|HAMAP-Rule:MF_00453}; Note=Binds 1 Mn(2+) ion per subunit. {ECO:0000255|HAMAP-Rule:MF_00453};',
       'COFACTOR: Name=Mn(2+); Xref=ChEBI:CHEBI:29035; Evidence={ECO:0000255|HAMAP-Rule:MF_00740}; Note=Binds 2 manganese ions. {ECO:0000255|HAMAP-Rule:MF_00740};',
       ...,
       'COFACTOR: Name=Fe(2+); Xref=ChEBI:CHEBI:29033; Evidence={ECO:0000269|PubMed:20945916};',
       'COFACTOR: Name=heme; Xref=ChEBI:CHEBI:30413; COFACTOR: Name=Mg(2+); Xref=ChEBI:CHEBI:18420;',
       'COFACTOR: Name=Zn(2+); Xref=ChEBI:CHEBI:29105; Evidence={ECO:0000269|Ref.5}; Note=Binds 2 Zn(2+) ions per subunit. {ECO:0000269|Ref.5};'],
      dtype=object)


# Context -> Subcellular location [CC] (65.36%), Pathway (26.76%)

In [51]:
df['Subcellular location [CC]'].unique()

array(['SUBCELLULAR LOCATION: Cell inner membrane {ECO:0000255|HAMAP-Rule:MF_01585}; Multi-pass membrane protein {ECO:0000255|HAMAP-Rule:MF_01585}.',
       'SUBCELLULAR LOCATION: Cytoplasm {ECO:0000255|HAMAP-Rule:MF_01109}.',
       'SUBCELLULAR LOCATION: Basolateral cell membrane; Multi-pass membrane protein.',
       ...,
       'SUBCELLULAR LOCATION: Golgi apparatus membrane {ECO:0000269|PubMed:10899003}; Single-pass type II membrane protein {ECO:0000269|PubMed:10899003}.',
       'SUBCELLULAR LOCATION: [Isoform 1]: Mitochondrion {ECO:0000269|PubMed:7599275}.; SUBCELLULAR LOCATION: [Isoform 2]: Mitochondrion {ECO:0000269|PubMed:7599275}. Nucleus inner membrane {ECO:0000269|PubMed:19602197, ECO:0000269|PubMed:7599275}; Peripheral membrane protein {ECO:0000269|PubMed:19602197, ECO:0000269|PubMed:7599275}; Nucleoplasmic side {ECO:0000269|PubMed:19602197, ECO:0000269|PubMed:7599275}. Note=Predominantly targeted to the nucleus. {ECO:0000269|PubMed:7599275}.',
       'SUBCELLULAR LOCATIO

In [52]:
df['Pathway'].unique()

array([nan,
       'PATHWAY: Amino-acid biosynthesis; L-arginine biosynthesis; L-arginine from L-ornithine and carbamoyl phosphate: step 1/3. {ECO:0000255|HAMAP-Rule:MF_01109}.',
       'PATHWAY: Purine metabolism; IMP biosynthesis via de novo pathway; 5-amino-1-(5-phospho-D-ribosyl)imidazole from N(2)-formyl-N(1)-(5-phospho-D-ribosyl)glycinamide: step 1/2. {ECO:0000255|HAMAP-Rule:MF_00421}.',
       ...,
       'PATHWAY: Porphyrin-containing compound metabolism; bacteriochlorophyll biosynthesis (light-independent). {ECO:0000305|PubMed:15090495}.',
       'PATHWAY: Protein modification; protein glycosylation. {ECO:0000269|PubMed:8877369}.',
       'PATHWAY: Lipopolysaccharide biosynthesis; LPS oligosaccharide biosynthesis. {ECO:0000303|PubMed:12676935, ECO:0000303|PubMed:16150700, ECO:0000303|PubMed:16800635, ECO:0000303|PubMed:17251184}.'],
      dtype=object)

# ID COLS -> Rhea ID (46.83%) / SABIO-RK (1.30%). -> reaction

In [53]:
df['Rhea ID'].unique()

array([nan, 'RHEA:19513', 'RHEA:24080', ..., 'RHEA:49180', 'RHEA:14257',
       'RHEA:28170'], dtype=object)

In [58]:
df['Catalytic activity'].unique()

array([nan,
       'CATALYTIC ACTIVITY: Reaction=carbamoyl phosphate + L-ornithine = L-citrulline + phosphate + H(+); Xref=Rhea:RHEA:19513, ChEBI:CHEBI:15378, ChEBI:CHEBI:43474, ChEBI:CHEBI:46911, ChEBI:CHEBI:57743, ChEBI:CHEBI:58228; EC=2.1.3.3; Evidence={ECO:0000255|HAMAP-Rule:MF_01109};',
       'CATALYTIC ACTIVITY: Reaction=tRNA(Trp) + L-tryptophan + ATP = L-tryptophyl-tRNA(Trp) + AMP + diphosphate + H(+); Xref=Rhea:RHEA:24080, Rhea:RHEA-COMP:9671, Rhea:RHEA-COMP:9705, ChEBI:CHEBI:15378, ChEBI:CHEBI:30616, ChEBI:CHEBI:33019, ChEBI:CHEBI:57912, ChEBI:CHEBI:78442, ChEBI:CHEBI:78535, ChEBI:CHEBI:456215; EC=6.1.1.2; Evidence={ECO:0000255|HAMAP-Rule:MF_00140};',
       ...,
       'CATALYTIC ACTIVITY: Reaction=3-O-(alpha-L-fucosyl)-L-threonyl-[EGF-like domain protein] + UDP-N-acetyl-alpha-D-glucosamine = 3-O-(N-acetyl-beta-D-glucosaminyl-(1->3)-alpha-L-fucosyl)-L-threonyl-[EGF-like domain protein] + UDP + H(+); Xref=Rhea:RHEA:70531, Rhea:RHEA-COMP:17922, Rhea:RHEA-COMP:17923, ChEBI:CHEB

In [59]:
# Find rows where Rhea ID is not null but Catalytic activity is null
missing_activity = df[df["Rhea ID"].notna() & df["Catalytic activity"].isna()]

print(f"Rows with Rhea ID but no Catalytic activity: {len(missing_activity)}")
missing_activity[["Rhea ID", "Catalytic activity"]].head(20)


Rows with Rhea ID but no Catalytic activity: 0


,Rhea ID,Catalytic activity


In [60]:
df = df.drop(columns=["Rhea ID", "SABIO-RK"])

# - Reactome (7.20%) / UniPathway (26.02%)  -> pathway

In [65]:
missing_kegg = df[df["Reactome"].notna() & df["Pathway"].isna()]
# count
print(f"Rows with Reactome but no Pathway: {len(missing_kegg)}")

missing_uni_pathway = df[df["UniPathway"].notna() & df["Pathway"].isna()]
print(f"Rows with UniPathway but no Pathway: {len(missing_uni_pathway)}")

Rows with Reactome but no Pathway: 15996
Rows with UniPathway but no Pathway: 0


In [ ]:
df['Pathway'].unique()

array([nan,
       'PATHWAY: Amino-acid biosynthesis; L-arginine biosynthesis; L-arginine from L-ornithine and carbamoyl phosphate: step 1/3. {ECO:0000255|HAMAP-Rule:MF_01109}.',
       'PATHWAY: Purine metabolism; IMP biosynthesis via de novo pathway; 5-amino-1-(5-phospho-D-ribosyl)imidazole from N(2)-formyl-N(1)-(5-phospho-D-ribosyl)glycinamide: step 1/2. {ECO:0000255|HAMAP-Rule:MF_00421}.',
       ...,
       'PATHWAY: Porphyrin-containing compound metabolism; bacteriochlorophyll biosynthesis (light-independent). {ECO:0000305|PubMed:15090495}.',
       'PATHWAY: Protein modification; protein glycosylation. {ECO:0000269|PubMed:8877369}.',
       'PATHWAY: Lipopolysaccharide biosynthesis; LPS oligosaccharide biosynthesis. {ECO:0000303|PubMed:12676935, ECO:0000303|PubMed:16150700, ECO:0000303|PubMed:16800635, ECO:0000303|PubMed:17251184}.'],
      dtype=object)

In [67]:
df["Reactome"].unique()

array([nan, 'R-DDI-204005;R-DDI-2132295;R-DDI-5694530;R-DDI-6798695;',
       'R-MMU-8951664;R-MMU-983168;', ..., 'R-CEL-5673001;',
       'R-DRE-5621480;R-DRE-913709;R-DRE-9696264;R-DRE-9696270;R-DRE-9696273;R-DRE-977068;',
       'R-SCE-193368;R-SCE-2046106;R-SCE-389887;R-SCE-390247;R-SCE-9033241;'],
      dtype=object)

# Topological domain (5.21%), Motif (4.68%), DNA binding (1.90%)

In [69]:
df['Topological domain'].unique()

array(['TOPO_DOM 1..17; /note="Periplasmic"; /evidence="ECO:0000255|HAMAP-Rule:MF_01585"; TOPO_DOM 39..52; /note="Cytoplasmic"; /evidence="ECO:0000255|HAMAP-Rule:MF_01585"; TOPO_DOM 74..90; /note="Periplasmic"; /evidence="ECO:0000255|HAMAP-Rule:MF_01585"; TOPO_DOM 112..144; /note="Cytoplasmic"; /evidence="ECO:0000255|HAMAP-Rule:MF_01585"; TOPO_DOM 166; /note="Periplasmic"; /evidence="ECO:0000255|HAMAP-Rule:MF_01585"; TOPO_DOM 188..226; /note="Cytoplasmic"; /evidence="ECO:0000255|HAMAP-Rule:MF_01585"; TOPO_DOM 248..256; /note="Periplasmic"; /evidence="ECO:0000255|HAMAP-Rule:MF_01585"; TOPO_DOM 278..280; /note="Cytoplasmic"; /evidence="ECO:0000255|HAMAP-Rule:MF_01585"; TOPO_DOM 302..304; /note="Periplasmic"; /evidence="ECO:0000255|HAMAP-Rule:MF_01585"; TOPO_DOM 326..343; /note="Cytoplasmic"; /evidence="ECO:0000255|HAMAP-Rule:MF_01585"; TOPO_DOM 365..366; /note="Periplasmic"; /evidence="ECO:0000255|HAMAP-Rule:MF_01585"; TOPO_DOM 388..397; /note="Cytoplasmic"; /evidence="ECO:0000255|HAMAP-

In [70]:
df['Motif']

0                                                       NaN
1                                                       NaN
2                                                       NaN
3                                                       NaN
4         MOTIF 14..22; /note="'HIGH' region"; /evidence...
                                ...                        
256685                                                  NaN
256686                                                  NaN
256687                                                  NaN
256688                                                  NaN
256689                                                  NaN
Name: Motif, Length: 256690, dtype: object

In [71]:
df['DNA binding'].unique()

array([nan,
       'DNA_BIND 246..305; /note="Homeobox"; /evidence="ECO:0000255|PROSITE-ProRule:PRU00108"',
       'DNA_BIND 34..110; /note="Nuclear receptor"; /evidence="ECO:0000255|PROSITE-ProRule:PRU00407"',
       ...,
       'DNA_BIND 343..402; /note="Homeobox"; /evidence="ECO:0000255|PROSITE-ProRule:PRU00108"',
       'DNA_BIND 51..70; /note="H-T-H motif"; /evidence="ECO:0000255|PROSITE-ProRule:PRU00307"',
       'DNA_BIND 42..61; /note="H-T-H motif"; /evidence="ECO:0000255|PROSITE-ProRule:PRU00411"'],
      dtype=object)

In [72]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 256690 entries, 0 to 256689
Data columns (total 25 columns):
 #   Column                              Non-Null Count   Dtype  
---  ------                              --------------   -----  
 0   Entry                               256690 non-null  object 
 1   name                                256690 non-null  object 
 2   sequence                            256690 non-null  object 
 3   function                            256690 non-null  object 
 4   Organism                            256681 non-null  object 
 5   Length                              256681 non-null  float64
 6   Active site                         55837 non-null   object 
 7   Catalytic activity                  132908 non-null  object 
 8   Binding site                        115900 non-null  object 
 9   Cofactor                            65251 non-null   object 
 10  DNA binding                         4886 non-null    object 
 11  EC number                 

In [73]:
df = df.drop(columns=["Temperature dependence", "pH dependence", "BRENDA"])

In [74]:
df.to_csv("QA_Final.csv", index=False)

# QA

In [2]:
import pandas as pd

In [29]:
qa_df = pd.read_csv("QA_Final.csv")

In [4]:
qa_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 256690 entries, 0 to 256689
Data columns (total 22 columns):
 #   Column                              Non-Null Count   Dtype  
---  ------                              --------------   -----  
 0   Entry                               256690 non-null  object 
 1   name                                256690 non-null  object 
 2   sequence                            256690 non-null  object 
 3   function                            256690 non-null  object 
 4   Organism                            256681 non-null  object 
 5   Length                              256681 non-null  float64
 6   Active site                         55837 non-null   object 
 7   Catalytic activity                  132908 non-null  object 
 8   Binding site                        115900 non-null  object 
 9   Cofactor                            65251 non-null   object 
 10  DNA binding                         4886 non-null    object 
 11  EC number                 

In [7]:
# Print 3 unique values from each column 
i = 1
for col in qa_df.columns:
    unique_vals = qa_df[col].dropna().unique()[:3]
    print(f"# {i}---{col}: {unique_vals}---")
    i += 1

# 1---Entry: ['B7LNJ1' 'Q6LG09' 'Q01345']---
# 2---name: ['LPLT_ESCF3' 'OTC_PHOPR' 'NHEB_ONCMY']---
# 3---sequence: ['MSESVHTNTSLWSKGMKAVIVAQFLSAFGDNALLFATLALLKAQFYPEWSQPVLQMVFVGAYILFAPFVGQVADSFAKGRVMMFANGLKLLGAASICFGFNPFVGYTLVGIGAAAYSPAKYGILGELTTGDKLVKANGLMEASTIAAILLGSVAGGVLADLHVLVALAACALAYAGAVAANIYIPKLAAARPGQSWNVLKMTCSFKSACTSLWQNGETRFSLVGTSLFWGAGVTLRFLLVLWVPVALGITDNATPTYLNAMVAIGIVLGAGAAAKLVTLETVSRCMPAGILIGVVVLFFSLQHELLPAYALLMLIGVLGGFFVVPLNALLQERGKKSVGAGNAIAVQNLGENSAMLLMLGIYSLAVLVGIPVVPIGIGFGTLFALAITALWIWQRRH'
 'MSFNLRNRNFLKLLDFTGKEIEHLIALAQDLKHAKYAGTEQQKLKGKNIALIFEKTSTRTRCAFEVAAHDQGAHVTYIGGGSQMGHKESTKDTARVLGRFYDGIEYRGFGQDVVETLGEHAGVPVWNGLTDEWHPTQIIADWMTMLEHGNGKRLNQMKLAYMGDAKNNMGNSLMVGAAKVGMEIRLVGPKAYWPDPALVAECNELCKISGGKIVITDNVQEGVDSVDFIYGDVWVSMGEPEELWATRINDLAPYQVNMSVITATQNPAVKYMHCLPAFHNDETRVGKKIEEKFNMKGLEVTEDVFESSYTICFDEAENRMHSIKAIMVATLGD'
 'MPAFSCAFPGCRRDLLVIVLVVFVGIGLPIEASAPAYQSHGTEGSHLTNITNTKKAFPVLAVNYEHVRKPFEIALWILLALLMKLGFHLIPRLSAVVPESCLLIVVGLLVGGLIKVIGEEPPVLDSQLFFLCLLPPIILDAGYF

In [11]:
show_coverage(qa_df)

,Col #,Column,Non-null count,Coverage %,Unique count
0,0,Entry,256690,100.00,256690
1,1,name,256690,100.00,256690
2,2,sequence,256690,100.00,256690
3,3,function,256690,100.00,52194
4,4,Organism,256681,100.00,7520
5,5,Length,256681,100.00,2305
19,19,Gene Ontology (molecular function),234925,91.52,24613
17,17,Gene Ontology (biological process),222047,86.50,36885
18,18,Gene Ontology (cellular component),215299,83.88,21760
15,15,Subcellular location [CC],167776,65.36,31886


In [14]:
qa_df['Organism']

0                    Escherichia fergusonii
1                  Photobacterium profundum
2                       Oncorhynchus mykiss
3                  Dictyostelium discoideum
4                      Agrobacterium fabrum
                        ...                
256685             Saccharomyces cerevisiae
256686           Mycobacterium tuberculosis
256687    Carboxydothermus hydrogenoformans
256688                          Danio rerio
256689              Drosophila melanogaster
Name: Organism, Length: 256690, dtype: object

# Pre-Processing

In [30]:
show_coverage(qa_df)

,Col #,Column,Non-null count,Coverage %,Unique count
0,0,Entry,256690,100.00,256690
1,1,name,256690,100.00,256690
2,2,sequence,256690,100.00,256690
3,3,function,256690,100.00,52194
4,4,Organism,256681,100.00,7520
5,5,Length,256681,100.00,2305
19,19,Gene Ontology (molecular function),234925,91.52,24613
17,17,Gene Ontology (biological process),222047,86.50,36885
18,18,Gene Ontology (cellular component),215299,83.88,21760
15,15,Subcellular location [CC],167776,65.36,31886


### Fixing missing Organism, length

In [32]:
# Fix length
import pandas as pd

# assume df is your dataframe

# fill missing Length values by computing sequence length
qa_df['Length'] = qa_df['Length'].fillna(qa_df['sequence'].str.len())

# sanity check: how many are still missing?
print("Missing Length after fix:", qa_df['Length'].isna().sum())

# optional: enforce integer type
qa_df['Length'] = qa_df['Length'].astype(int)

Missing Length after fix: 0


In [33]:
qa_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 256690 entries, 0 to 256689
Data columns (total 22 columns):
 #   Column                              Non-Null Count   Dtype 
---  ------                              --------------   ----- 
 0   Entry                               256690 non-null  object
 1   name                                256690 non-null  object
 2   sequence                            256690 non-null  object
 3   function                            256690 non-null  object
 4   Organism                            256681 non-null  object
 5   Length                              256690 non-null  int64 
 6   Active site                         55837 non-null   object
 7   Catalytic activity                  132908 non-null  object
 8   Binding site                        115900 non-null  object
 9   Cofactor                            65251 non-null   object
 10  DNA binding                         4886 non-null    object
 11  EC number                           141

In [35]:
missing_orgs = qa_df[qa_df['Organism'].isna()][['Entry', 'sequence']]

print("Number of rows missing Organism:", len(missing_orgs))
print(missing_orgs)

Number of rows missing Organism: 9
             Entry                                           sequence
4733    A0A0U1LSP0  MASQSAHLVKTHQLWELLRHRTKTGSYELDGDSLHIADVVATAQQT...
46027       Q6GMV1  MERSAFMELDAGSRLVMHLREWPALLVSSTGWTEFEQLTLDGHNLP...
54429       P79371  MAAQKDLWDAIVIGAGIQGCFTVYHLVKHRKRILLLEQFFLPHSRG...
115258      P07631  MMSDYNWFEGIPFPAISYQREILEDIRNKFVVKEEDLLILTYPKSG...
170978      P36185  AAGIRIASAIDFNTWASKNNKHFTAIEKLRRRAIFNMNAKFVDSFN...
189684  A0A0B4J2F2  MVIMSEFSADPAGQGQGQQKPLRVGFYDIERTLGKGNFAVVKLARH...
207357      Q06964  LAIANAIDFNTWAANNNKHFTAVEALRRRAIFNMNARFVAEFNKKG...
245018      P0DUJ1  MKFLWAALVVTLLAGCQADVEEEVKLGQEPDRWQAKQPWEQALGRF...
252773      Q8WV35  MYSSGWPAGAAEPRHGRGRELAQALGCMHGAPSQLASLSLAHCSSL...


In [36]:
qa_df.head(2)

,Entry,name,sequence,function,Organism,Length,Active site,Catalytic activity,Binding site,Cofactor,...,Pathway,Reactome,UniPathway,Subcellular location [CC],Domain [FT],Gene Ontology (biological process),Gene Ontology (cellular component),Gene Ontology (molecular function),Motif,Topological domain
0,B7LNJ1,LPLT_ESCF3,MSESVHTNTSLWSKGMKAVIVAQFLSAFGDNALLFATLALLKAQFY...,Catalyzes the facilitated diffusion of 2-acyl-...,Escherichia fergusonii,397,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,SUBCELLULAR LOCATION: Cell inner membrane {ECO...,NaN,NaN,plasma membrane [GO:0005886],lysophospholipid:sodium symporter activity [GO...,NaN,"TOPO_DOM 1..17; /note=""Periplasmic""; /evidence..."
1,Q6LG09,OTC_PHOPR,MSFNLRNRNFLKLLDFTGKEIEHLIALAQDLKHAKYAGTEQQKLKG...,Reversibly catalyzes the transfer of the carba...,Photobacterium profundum,333,NaN,CATALYTIC ACTIVITY: Reaction=carbamoyl phospha...,"BINDING 57..60; /ligand=""carbamoyl phosphate"";...",NaN,...,PATHWAY: Amino-acid biosynthesis; L-arginine b...,NaN,UPA00068;,SUBCELLULAR LOCATION: Cytoplasm {ECO:0000255|H...,NaN,arginine biosynthetic process via ornithine [G...,cytoplasm [GO:0005737],amino acid binding [GO:0016597]; ornithine car...,NaN,NaN


In [37]:
old_df = pd.read_csv("Prot2text_UniProt.csv")

In [38]:
old_df['taxon'].unique()

array(['Escherichia', 'Photobacterium', 'Oncorhynchus', ..., 'Polyorchis',
       'Sulfitobacter', 'Lobophytum'], dtype=object)

In [39]:
qa_df['Organism'].unique()

array(['Escherichia fergusonii', 'Photobacterium profundum',
       'Oncorhynchus mykiss', ..., 'Sulfitobacter sp.',
       'Lobophytum crassum', 'Streptomyces pratensis'], dtype=object)

In [40]:
import pandas as pd

# assume you already have:
# df = your new dataframe (with missing Organism for 9 entries)
# old_df = previous dataset with 'taxon' column

# list of the 9 missing Entry IDs
missing_entries = [
    "A0A0U1LSP0", "Q6GMV1", "P79371", "P07631", "P36185",
    "A0A0B4J2F2", "Q06964", "P0DUJ1", "Q8WV35"
]

# search them in old_df
lookup = old_df[old_df['Entry'].isin(missing_entries)][['Entry', 'taxon']]
lookup


,Entry,taxon
4733,A0A0U1LSP0,Talaromyces sect. Islandici
46027,Q6GMV1,Homo
54429,P79371,Oryctolagus
115258,P07631,Rattus
170978,P36185,Entamoeba
189684,A0A0B4J2F2,Homo
207357,Q06964,Entamoeba
245018,P0DUJ1,Pteropus
252773,Q8WV35,Homo


In [41]:
# build dictionary: short taxon -> list of full organism names in qa_df
taxon_to_full = {}
for taxon in lookup['taxon'].unique():
    matches = qa_df[qa_df['Organism'].str.contains(taxon, case=False, na=False)]['Organism'].unique().tolist()
    taxon_to_full[taxon] = matches

print("Mapping taxon to full Organism names found in qa_df:")
for k, v in taxon_to_full.items():
    print(f"{k} -> {v}")


Mapping taxon to full Organism names found in qa_df:
Talaromyces sect. Islandici -> []
Homo -> ['Homo sapiens', 'Trichomonas vaginalis', 'Stenotrophomonas maltophilia', 'Hyphomonas neptunium', 'Xanthomonas axonopodis pv. citri', 'Syntrophomonas wolfei subsp. wolfei', 'Xanthomonas oryzae pv. oryzae', 'Xanthomonas campestris pv. campestris', 'Xanthomonas campestris pv. phaseoli', 'Xanthomonas euvesicatoria pv. vesicatoria', 'Phomopsis amygdali', 'Xanthomonas campestris pv. cyanopsidis', 'Homoroselaps lacteus', 'Homoeomma sp.', 'Xanthomonas campestris pv. juglandis', 'Xanthomonas campestris', 'Xanthomonas euvesicatoria', 'Xanthomonas albilineans', 'Abies homolepis', 'Thomomys mazama', 'Xanthomonas campestris pv. translucens', 'Pseudoxanthomonas mexicana', 'Tritrichomonas foetus', 'Thomomys talpoides', 'Homo sapiens neanderthalensis', 'Xanthomonas citri', 'Xanthomonas campestris pv. glycines', 'Thomomys bulbivorus', 'Xanthomonas vasicola', 'Xanthomonas oryzae pv. oryzicola', 'Xanthomonas c

In [43]:
# dictionary for safe auto-fills
manual_map = {
    "Oryctolagus": "Oryctolagus cuniculus",
    "Homo": "Homo sapiens",  
    "Rattus": "Rattus sp.",
    "Entamoeba": "Entamoeba sp.",
    "Pteropus": "Pteropus sp.",
    "Talaromyces sect. Islandici": "Talaromyces sect. Islandici"
}

# patch Organism values for missing entries
for entry, taxon in zip(lookup['Entry'], lookup['taxon']):
    if pd.isna(qa_df.loc[qa_df['Entry'] == entry, 'Organism']).any():
        qa_df.loc[qa_df['Entry'] == entry, 'Organism'] = manual_map.get(taxon, taxon)

# verify all fixed
print(qa_df.loc[qa_df['Entry'].isin(missing_entries), ['Entry', 'Organism']])

             Entry                     Organism
4733    A0A0U1LSP0  Talaromyces sect. Islandici
46027       Q6GMV1                 Homo sapiens
54429       P79371        Oryctolagus cuniculus
115258      P07631                   Rattus sp.
170978      P36185                Entamoeba sp.
189684  A0A0B4J2F2                 Homo sapiens
207357      Q06964                Entamoeba sp.
245018      P0DUJ1                 Pteropus sp.
252773      Q8WV35                 Homo sapiens


In [45]:
qa_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 256690 entries, 0 to 256689
Data columns (total 22 columns):
 #   Column                              Non-Null Count   Dtype 
---  ------                              --------------   ----- 
 0   Entry                               256690 non-null  object
 1   name                                256690 non-null  object
 2   sequence                            256690 non-null  object
 3   function                            256690 non-null  object
 4   Organism                            256690 non-null  object
 5   Length                              256690 non-null  int64 
 6   Active site                         55837 non-null   object
 7   Catalytic activity                  132908 non-null  object
 8   Binding site                        115900 non-null  object
 9   Cofactor                            65251 non-null   object
 10  DNA binding                         4886 non-null    object
 11  EC number                           141

In [50]:
# Make sure length col is coorrct. 
# recompute the length from the sequence, and overwrite the length col. Keep it int

qa_df['length'] = qa_df['sequence'].str.len().astype(int)

In [51]:
qa_df.length

0          397
1          333
2          759
3          448
4          354
          ... 
256685     253
256686     301
256687     535
256688     212
256689    1365
Name: length, Length: 256690, dtype: int64

In [52]:
qa_df.to_csv("QA_Final_v2.csv", index=False)

### Active site, Binding site

- Active site: Marks key catalytic residues directly responsible for the chemical reaction.
- Binding site: Marks residues interacting with ligands/cofactors/substrates/products.

In [53]:
qa_df['Active site'].unique()

array([nan,
       'ACT_SITE 262; /evidence="ECO:0000255|PROSITE-ProRule:PRU10088"; ACT_SITE 397; /evidence="ECO:0000255|PROSITE-ProRule:PRU10088"; ACT_SITE 415; /evidence="ECO:0000255|PROSITE-ProRule:PRU10088"',
       'ACT_SITE 154; /note="Nucleophile"; /evidence="ECO:0000269|PubMed:15081808"; ACT_SITE 215; /note="Charge relay system"; /evidence="ECO:0000305|PubMed:15103133"; ACT_SITE 268; /note="Charge relay system"; /evidence="ECO:0000305|PubMed:15103133"',
       ...,
       'ACT_SITE 246; /note="Nucleophile"; /evidence="ECO:0000305|PubMed:16797589"',
       'ACT_SITE 259; /note="Charge relay system"; /evidence="ECO:0000250"; ACT_SITE 282; /note="Charge relay system"; /evidence="ECO:0000250"; ACT_SITE 333; /note="Charge relay system"; /evidence="ECO:0000250"',
       'ACT_SITE 327; /evidence="ECO:0000250"'], dtype=object)

In [54]:
qa_df['Binding site'].unique()

array([nan,
       'BINDING 57..60; /ligand="carbamoyl phosphate"; /ligand_id="ChEBI:CHEBI:58228"; /evidence="ECO:0000255|HAMAP-Rule:MF_01109"; BINDING 83; /ligand="carbamoyl phosphate"; /ligand_id="ChEBI:CHEBI:58228"; /evidence="ECO:0000255|HAMAP-Rule:MF_01109"; BINDING 107; /ligand="carbamoyl phosphate"; /ligand_id="ChEBI:CHEBI:58228"; /evidence="ECO:0000255|HAMAP-Rule:MF_01109"; BINDING 134..137; /ligand="carbamoyl phosphate"; /ligand_id="ChEBI:CHEBI:58228"; /evidence="ECO:0000255|HAMAP-Rule:MF_01109"; BINDING 168; /ligand="L-ornithine"; /ligand_id="ChEBI:CHEBI:46911"; /evidence="ECO:0000255|HAMAP-Rule:MF_01109"; BINDING 232; /ligand="L-ornithine"; /ligand_id="ChEBI:CHEBI:46911"; /evidence="ECO:0000255|HAMAP-Rule:MF_01109"; BINDING 236..237; /ligand="L-ornithine"; /ligand_id="ChEBI:CHEBI:46911"; /evidence="ECO:0000255|HAMAP-Rule:MF_01109"; BINDING 274..275; /ligand="carbamoyl phosphate"; /ligand_id="ChEBI:CHEBI:58228"; /evidence="ECO:0000255|HAMAP-Rule:MF_01109"; BINDING 319; /ligan

In [60]:
import re
import pandas as pd

def parse_active_site_json(s):
    if pd.isna(s):
        return None
    sites = []
    for chunk in s.split("ACT_SITE"):
        chunk = chunk.strip()
        if not chunk:
            continue
        # position
        pos_match = re.match(r"(\d+)", chunk)
        pos = int(pos_match.group(1)) if pos_match else None
        # note (can contain semicolons etc.)
        note_match = re.search(r'/note="([^"]+)"', chunk)
        note = note_match.group(1) if note_match else None
        # evidence
        eco_matches = re.findall(r'ECO:\d+', chunk)
        pmid_matches = re.findall(r'PubMed:\d+', chunk)
        evidence = eco_matches + pmid_matches
        sites.append({
            "pos": pos,
            "note": note,
            "evidence": evidence
        })
    return sites

# Apply to your dataframe
qa_df['Active_site_json'] = qa_df['Active site'].apply(parse_active_site_json)

# Example check
import json
print(json.dumps(qa_df['Active_site_json'].dropna().iloc[0], indent=2))

[
  {
    "pos": 262,
    "note": null,
    "evidence": [
      "ECO:0000255"
    ]
  },
  {
    "pos": 397,
    "note": null,
    "evidence": [
      "ECO:0000255"
    ]
  },
  {
    "pos": 415,
    "note": null,
    "evidence": [
      "ECO:0000255"
    ]
  }
]


In [61]:
qa_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 256690 entries, 0 to 256689
Data columns (total 24 columns):
 #   Column                              Non-Null Count   Dtype 
---  ------                              --------------   ----- 
 0   Entry                               256690 non-null  object
 1   name                                256690 non-null  object
 2   sequence                            256690 non-null  object
 3   function                            256690 non-null  object
 4   Organism                            256690 non-null  object
 5   Length                              256690 non-null  int64 
 6   Active site                         55837 non-null   object
 7   Catalytic activity                  132908 non-null  object
 8   Binding site                        115900 non-null  object
 9   Cofactor                            65251 non-null   object
 10  DNA binding                         4886 non-null    object
 11  EC number                           141

In [69]:
# download entry, active_site_json for entries with active_site_json not null tsv
qa_df[qa_df['Active_site_json'].notna()][['Entry', 'Active site','Active_site_json']].to_csv("temp/QA_Active_Sites.tsv", sep='\t', index=False)

In [77]:
import pandas as pd
import re
import json
import random


# ---- Parser ----
pos_pat = re.compile(r'^\s*(\d+)(?:\.\.(\d+))?')
ligand_pat = re.compile(r'/ligand="([^"]+)"')
ligandid_pat = re.compile(r'/ligand_id="([^"]+)"')
note_pat = re.compile(r'/note="([^"]+)"')
evidence_pat = re.compile(r'/evidence="([^"]+)"')
pmid_inline_pat = re.compile(r'PubMed:\d+')
eco_inline_pat = re.compile(r'ECO:\d+')

def parse_binding_sites_to_json(s: str):
    if pd.isna(s) or not isinstance(s, str) or not s.strip():
        return None
    blocks = s.split("BINDING")
    out = []
    for blk in blocks:
        blk = blk.strip()
        if not blk:
            continue
        # positions
        m = pos_pat.match(blk)
        if not m:
            continue
        start = int(m.group(1))
        end = int(m.group(2)) if m.group(2) else start
        # ligand fields
        lig_name = (ligand_pat.search(blk).group(1)
                    if ligand_pat.search(blk) else None)
        lig_id = (ligandid_pat.search(blk).group(1)
                  if ligandid_pat.search(blk) else None)
        # note
        note_m = note_pat.search(blk)
        note = note_m.group(1) if note_m else None
        # evidence
        evidence_tokens = []
        m5 = evidence_pat.search(blk)
        if m5:
            evidence_tokens.extend([tok.strip() for tok in m5.group(1).split("|") if tok.strip()])
        evidence_tokens.extend(eco_inline_pat.findall(blk))
        evidence_tokens.extend(pmid_inline_pat.findall(blk))
        # de-dupe
        seen, ev_list = set(), []
        for tok in evidence_tokens:
            if tok not in seen:
                seen.add(tok); ev_list.append(tok)
        out.append({
            "start": start,
            "end": end,
            "ligand_name": lig_name,
            "ligand_id": lig_id,
            "note": note,
            "evidence": ev_list
        })
    return out if out else None

# Apply + strict JSON
qa_df['Binding_site_parsed'] = qa_df['Binding site'].apply(parse_binding_sites_to_json)
qa_df['Binding_site_json'] = qa_df['Binding_site_parsed'].apply(
    lambda obj: None if obj is None else json.dumps(obj, ensure_ascii=False)
)

# Validate
def is_valid_json_list(s):
    if s is None:
        return True
    try:
        parsed = json.loads(s)
        if not isinstance(parsed, list): return False
        for d in parsed:
            if not isinstance(d, dict): return False
            for k in ["start", "end", "ligand_name", "ligand_id", "note", "evidence"]:
                if k not in d: return False
        return True
    except Exception:
        return False

qa_df['valid'] = qa_df['Binding_site_json'].apply(is_valid_json_list)
print("Validation:", qa_df['valid'].sum(), "/", len(qa_df), "valid")

# Save cleaned TSV
qa_df[['Entry','Binding site','Binding_site_json']].to_csv(
    "temp/QA_Binding_Sites_with_JSON.tsv", sep="\t", index=False
)

# Also save 30 manual samples (original + JSON)
import sys
import random
sample_indices = random.sample(range(len(qa_df)), k=min(30, len(qa_df)))
lines = [f"Total rows: {len(qa_df)}\n"]
for idx in sample_indices:
    orig = qa_df.iloc[idx]['Binding site']
    js = qa_df.iloc[idx]['Binding_site_json']
    lines.append(f"[{idx}] Binding site:\n{orig}\nParsed JSON:\n{js}\n")
with open("temp/binding_site_parsing_samples.txt", "w", encoding="utf-8") as f:
    f.write("\n".join(lines))

Validation: 256690 / 256690 valid


In [78]:
qa_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 256690 entries, 0 to 256689
Data columns (total 27 columns):
 #   Column                              Non-Null Count   Dtype 
---  ------                              --------------   ----- 
 0   Entry                               256690 non-null  object
 1   name                                256690 non-null  object
 2   sequence                            256690 non-null  object
 3   function                            256690 non-null  object
 4   Organism                            256690 non-null  object
 5   Length                              256690 non-null  int64 
 6   Active site                         55837 non-null   object
 7   Catalytic activity                  132908 non-null  object
 8   Binding site                        115900 non-null  object
 9   Cofactor                            65251 non-null   object
 10  DNA binding                         4886 non-null    object
 11  EC number                           141

In [81]:
qa_df = qa_df.drop(columns=["Binding_site_parsed", "valid"])

In [84]:
qa_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 256690 entries, 0 to 256689
Data columns (total 25 columns):
 #   Column                              Non-Null Count   Dtype 
---  ------                              --------------   ----- 
 0   Entry                               256690 non-null  object
 1   name                                256690 non-null  object
 2   sequence                            256690 non-null  object
 3   function                            256690 non-null  object
 4   Organism                            256690 non-null  object
 5   Length                              256690 non-null  int64 
 6   Active site                         55837 non-null   object
 7   Catalytic activity                  132908 non-null  object
 8   Binding site                        115900 non-null  object
 9   Cofactor                            65251 non-null   object
 10  DNA binding                         4886 non-null    object
 11  EC number                           141

### Catalytic activity

In [87]:
import pandas as pd
import re
import json

# --- Patterns ---
reaction_iter_pat = re.compile(r'Reaction=')
field_reaction_pat = re.compile(r'Reaction=([^;]+);')
ec_pat = re.compile(r'EC=([^;]+);')
evidence_pat = re.compile(r'Evidence=\{([^}]+)\};')

rhea_reac_token = re.compile(r'RHEA:\d+')
rhea_comp_token = re.compile(r'RHEA-COMP:\d+')
chebi_any_token = re.compile(r'(?:ChEBI:)?(CHEBI:\d+)')
eco_token = re.compile(r'ECO:\d+')
pmid_token = re.compile(r'PubMed:\d+')

def _dedup(seq):
    seen = set(); outl = []
    for t in seq:
        if t not in seen:
            seen.add(t); outl.append(t)
    return outl

def parse_catalytic_activity_to_json(s: str):
    """Return list of reaction blocks with reaction text, RHEA/RHEA-COMP/CHEBI/EC/evidence."""
    if pd.isna(s) or not isinstance(s, str) or not s.strip():
        return None
    text = s.strip()

    # Segment by successive Reaction= occurrences
    blocks = [m.start() for m in reaction_iter_pat.finditer(text)]
    if not blocks:
        blocks = [0]
    blocks.append(len(text))

    out = []
    for i in range(len(blocks)-1):
        seg = text[blocks[i]:blocks[i+1]]

        # Reaction text
        m_reac = field_reaction_pat.search(seg)
        reaction = m_reac.group(1).strip() if m_reac else None

        # IDs from the whole segment
        rhea_ids = _dedup(rhea_reac_token.findall(seg))
        rhea_comp_ids = _dedup(rhea_comp_token.findall(seg))
        chebi_ids = _dedup([m.group(1) for m in chebi_any_token.finditer(seg)])

        # EC numbers
        ec_numbers = []
        m_ec = ec_pat.search(seg)
        if m_ec:
            ec_numbers = [tok.strip() for tok in m_ec.group(1).split(",") if tok.strip()]

        # Evidence (block + inline)
        evidence = []
        m_ev = evidence_pat.search(seg)
        if m_ev:
            evidence.extend([tok.strip() for tok in m_ev.group(1).split("|") if tok.strip()])
        for tok in eco_token.findall(seg):
            if tok not in evidence:
                evidence.append(tok)
        for tok in pmid_token.findall(seg):
            if tok not in evidence:
                evidence.append(tok)

        if not (reaction or rhea_ids or rhea_comp_ids or chebi_ids or ec_numbers or evidence):
            continue

        out.append({
            "reaction": reaction,
            "rhea_ids": rhea_ids,
            "rhea_comp_ids": rhea_comp_ids,
            "chebi_ids": chebi_ids,
            "ec_numbers": ec_numbers,
            "evidence": evidence
        })

    return out if out else None

# --- Apply on your dataframe df ---
# df = pd.read_csv("YOUR_FILE.tsv", sep="\t")
df = qa_df.copy()
df['Catalytic_Activity_json'] = df['Catalytic activity'].apply(
    lambda s: None if (obj := parse_catalytic_activity_to_json(s)) is None else json.dumps(obj, ensure_ascii=False)
)


In [88]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 256690 entries, 0 to 256689
Data columns (total 26 columns):
 #   Column                              Non-Null Count   Dtype 
---  ------                              --------------   ----- 
 0   Entry                               256690 non-null  object
 1   name                                256690 non-null  object
 2   sequence                            256690 non-null  object
 3   function                            256690 non-null  object
 4   Organism                            256690 non-null  object
 5   Length                              256690 non-null  int64 
 6   Active site                         55837 non-null   object
 7   Catalytic activity                  132908 non-null  object
 8   Binding site                        115900 non-null  object
 9   Cofactor                            65251 non-null   object
 10  DNA binding                         4886 non-null    object
 11  EC number                           141

In [91]:
# save active site, active site json, binding site, binding site json, catalytic activity, catalytic activity json into a tsv. total three csv. 3 cols per csv. only non null rows
df[df['Active site'].notna()][['Entry', 'Active site','Active_site_json']].to_csv("temp/QA_Active_Sites.tsv", sep='\t', index=False)
df[df['Binding site'].notna()][['Entry','Binding site','Binding_site_json']].to_csv("temp/QA_Binding_Sites_with_JSON.tsv", sep="\t", index=False)
df[df['Catalytic activity'].notna()][['Entry','Catalytic activity','Catalytic_Activity_json']].to_csv("temp/QA_Catalytic_Activities_with_JSON.tsv", sep="\t", index=False)


In [92]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 256690 entries, 0 to 256689
Data columns (total 26 columns):
 #   Column                              Non-Null Count   Dtype 
---  ------                              --------------   ----- 
 0   Entry                               256690 non-null  object
 1   name                                256690 non-null  object
 2   sequence                            256690 non-null  object
 3   function                            256690 non-null  object
 4   Organism                            256690 non-null  object
 5   Length                              256690 non-null  int64 
 6   Active site                         55837 non-null   object
 7   Catalytic activity                  132908 non-null  object
 8   Binding site                        115900 non-null  object
 9   Cofactor                            65251 non-null   object
 10  DNA binding                         4886 non-null    object
 11  EC number                           141

### Cofactor, DNA binding

In [93]:
df['Cofactor'].unique()

array([nan,
       'COFACTOR: Name=Mn(2+); Xref=ChEBI:CHEBI:29035; Evidence={ECO:0000255|HAMAP-Rule:MF_00453}; Note=Binds 1 Mn(2+) ion per subunit. {ECO:0000255|HAMAP-Rule:MF_00453};',
       'COFACTOR: Name=Mn(2+); Xref=ChEBI:CHEBI:29035; Evidence={ECO:0000255|HAMAP-Rule:MF_00740}; Note=Binds 2 manganese ions. {ECO:0000255|HAMAP-Rule:MF_00740};',
       ...,
       'COFACTOR: Name=Fe(2+); Xref=ChEBI:CHEBI:29033; Evidence={ECO:0000269|PubMed:20945916};',
       'COFACTOR: Name=heme; Xref=ChEBI:CHEBI:30413; COFACTOR: Name=Mg(2+); Xref=ChEBI:CHEBI:18420;',
       'COFACTOR: Name=Zn(2+); Xref=ChEBI:CHEBI:29105; Evidence={ECO:0000269|Ref.5}; Note=Binds 2 Zn(2+) ions per subunit. {ECO:0000269|Ref.5};'],
      dtype=object)

In [94]:
# save all the cofactor rows with non null cofactor
df[df['Cofactor'].notna()][['Entry','Cofactor']].to_csv("temp/QA_Cofactors.tsv", sep="\t", index=False)

# dna binding
df[df['DNA binding'].notna()][['Entry','DNA binding']].to_csv("temp/QA_DNA_Binding.tsv", sep="\t", index=False)

In [ ]:
import pandas as pd
import re
import json

# Regex patterns
block_pat = re.compile(r'Name=([^;]+);')
evidence_pat = re.compile(r'Evidence=\{([^}]+)\};')
chebi_token = re.compile(r'(?:ChEBI:)?(CHEBI:\d+)')
eco_token = re.compile(r'ECO:\d+')
pmid_token = re.compile(r'PubMed:\d+')

def parse_cofactor_to_json(s: str):
    if pd.isna(s) or not isinstance(s, str) or not s.strip():
        return None
    text = s.strip()
    # Split into blocks
    blocks = [m.start() for m in block_pat.finditer(text)]
    if not blocks:
        blocks = [0]
    blocks.append(len(text))

    out = []
    for i in range(len(blocks)-1):
        seg = text[blocks[i]:blocks[i+1]]

        # Name
        name = None
        m_name = block_pat.search(seg)
        if m_name:
            name = m_name.group(1).strip()

        # ChEBI IDs
        chebi_ids = [m.group(1) for m in chebi_token.finditer(seg)]
        chebi_ids = list(dict.fromkeys(chebi_ids))

        # Evidence
        evidence = []
        m_ev = evidence_pat.search(seg)
        if m_ev:
            evidence.extend([tok.strip() for tok in m_ev.group(1).split("|") if tok.strip()])
        for tok in eco_token.findall(seg):
            if tok not in evidence:
                evidence.append(tok)
        for tok in pmid_token.findall(seg):
            if tok not in evidence:
                evidence.append(tok)

        if not (name or chebi_ids or evidence):
            continue

        out.append({
            "name": name,
            "chebi_ids": chebi_ids,
            "evidence": evidence
        })
    return out if out else None

# Apply to dataset
df['Cofactor_json'] = df['Cofactor'].apply(
    lambda s: None if (obj := parse_cofactor_to_json(s)) is None else json.dumps(obj, ensure_ascii=False)
)


In [98]:
# non null cofactor rows
# Save
df[df['Cofactor'].notna()][['Entry', 'Cofactor', 'Cofactor_json']].to_csv("temp/QA_Cofactors_with_JSON.tsv", sep="\t", index=False)


df[df['Active site'].notna()][['Entry', 'Active site', 'Active_site_json']].to_csv("temp/QA_Active_Sites_with_JSON.tsv", sep='\t', index=False)

In [100]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 256690 entries, 0 to 256689
Data columns (total 27 columns):
 #   Column                              Non-Null Count   Dtype 
---  ------                              --------------   ----- 
 0   Entry                               256690 non-null  object
 1   name                                256690 non-null  object
 2   sequence                            256690 non-null  object
 3   function                            256690 non-null  object
 4   Organism                            256690 non-null  object
 5   Length                              256690 non-null  int64 
 6   Active site                         55837 non-null   object
 7   Catalytic activity                  132908 non-null  object
 8   Binding site                        115900 non-null  object
 9   Cofactor                            65251 non-null   object
 10  DNA binding                         4886 non-null    object
 11  EC number                           141

In [101]:
import pandas as pd
import re
import json

pos_pat = re.compile(r'(\d+)(?:\.\.(\d+))?')
note_pat = re.compile(r'/note="([^"]+)"')
evidence_pat = re.compile(r'/evidence="([^"]+)"')
eco_token = re.compile(r'ECO:\d+')
pmid_token = re.compile(r'PubMed:\d+')

def parse_dna_binding_to_json(s: str):
    if pd.isna(s) or not isinstance(s, str) or not s.strip():
        return None
    text = s.strip()
    blocks = text.split("DNA_BIND")
    out = []
    for blk in blocks:
        blk = blk.strip()
        if not blk:
            continue
        # Position(s)
        m = pos_pat.search(blk)
        if not m:
            continue
        start = int(m.group(1))
        end = int(m.group(2)) if m.group(2) else start

        # Note
        note = None
        m_note = note_pat.search(blk)
        if m_note:
            note = m_note.group(1)

        # Evidence
        evidence = []
        m_ev = evidence_pat.search(blk)
        if m_ev:
            evidence.extend([tok.strip() for tok in m_ev.group(1).split("|") if tok.strip()])
        for tok in eco_token.findall(blk):
            if tok not in evidence:
                evidence.append(tok)
        for tok in pmid_token.findall(blk):
            if tok not in evidence:
                evidence.append(tok)

        out.append({
            "start": start,
            "end": end,
            "note": note,
            "evidence": evidence
        })
    return out if out else None


df['DNA_binding_json'] = df['DNA binding'].apply(
    lambda s: None if (obj := parse_dna_binding_to_json(s)) is None else json.dumps(obj, ensure_ascii=False)
)

# Save non null dna binding rows
df[df['DNA binding'].notna()][['Entry', 'DNA binding', 'DNA_binding_json']].to_csv("temp/QA_DNA_Binding_with_JSON.tsv", sep='\t', index=False)

In [102]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 256690 entries, 0 to 256689
Data columns (total 28 columns):
 #   Column                              Non-Null Count   Dtype 
---  ------                              --------------   ----- 
 0   Entry                               256690 non-null  object
 1   name                                256690 non-null  object
 2   sequence                            256690 non-null  object
 3   function                            256690 non-null  object
 4   Organism                            256690 non-null  object
 5   Length                              256690 non-null  int64 
 6   Active site                         55837 non-null   object
 7   Catalytic activity                  132908 non-null  object
 8   Binding site                        115900 non-null  object
 9   Cofactor                            65251 non-null   object
 10  DNA binding                         4886 non-null    object
 11  EC number                           141

### pathway

In [107]:
import pandas as pd
import re
import json

# ----- Patterns -----
PATHWAY_SPLIT = re.compile(r'PATHWAY:\s*', flags=re.IGNORECASE)
EVID_BLOCK    = re.compile(r'\{([^}]*)\}')                       # contents inside {...}
STEP_RX       = re.compile(r'\bstep\s+(\d+)\s*/\s*(\d+)\b', flags=re.IGNORECASE)

def parse_pathway_to_json(s: str):
    """
    Parse a UniProt-style PATHWAY field into a JSON-ready list of segments.
    Each segment:
      - levels: list[str]  (semicolon-separated hierarchy, sans 'step X/Y' and evidence)
      - step_index: int|None
      - step_total: int|None
      - evidence: list[str] (split on '|' or ',' within {...})
    """
    if pd.isna(s) or not isinstance(s, str) or not s.strip():
        return None

    text = s.strip().rstrip(".")
    segments = [p for p in PATHWAY_SPLIT.split(text) if p.strip()]
    out = []

    for seg in segments:
        seg = seg.strip().rstrip(".")

        # Evidence in {...}
        evidence = []
        m_e = EVID_BLOCK.search(seg)
        if m_e:
            ev_text = m_e.group(1)
            # split on pipe or comma; normalize whitespace
            evidence = [t.strip() for t in re.split(r'[|,]', ev_text) if t.strip()]
            # remove evidence block from segment
            seg = seg[:m_e.start()].strip().rstrip(".")

        # Hierarchy levels
        levels = [p.strip() for p in seg.split(';') if p.strip()]

        # step X/Y at tail of last level
        step_index = step_total = None
        if levels:
            last = levels[-1]
            m_s = STEP_RX.search(last)
            if m_s:
                step_index = int(m_s.group(1))
                step_total = int(m_s.group(2))
                # remove " : step X/Y" or " step X/Y" from last level
                cleaned_last = re.sub(r'[:\s]*step\s+\d+\s*/\s*\d+\s*$', '', last, flags=re.IGNORECASE).rstrip(".").strip()
                levels[-1] = cleaned_last

        out.append({
            "levels": levels,
            "step_index": step_index,
            "step_total": step_total,
            "evidence": evidence
        })

    return out if out else None

# ----- Apply to your dataset -----
df['Pathway_json'] = df['Pathway'].apply(
    lambda s: None if (obj := parse_pathway_to_json(s)) is None else json.dumps(obj, ensure_ascii=False)
)

# Optional: quick validation on a random sample (value-level)
# (same logic as parser; compare parsed vs re-extracted)
def extract_reference_signals(raw: str):
    if pd.isna(raw) or not isinstance(raw, str) or not raw.strip():
        return []
    text = raw.strip().rstrip(".")
    segs = [p for p in PATHWAY_SPLIT.split(text) if p.strip()]
    sigs = []
    for seg in segs:
        seg = seg.strip().rstrip(".")
        evidence = []
        m = EVID_BLOCK.search(seg)
        if m:
            evidence = [t.strip() for t in re.split(r'[|,]', m.group(1)) if t.strip()]
            seg = seg[:m.start()].strip().rstrip(".")
        levels = [p.strip() for p in seg.split(';') if p.strip()]
        s_idx = s_tot = None
        if levels:
            last = levels[-1]
            m_s = STEP_RX.search(last)
            if m_s:
                s_idx, s_tot = int(m_s.group(1)), int(m_s.group(2))
                levels[-1] = re.sub(r'[:\s]*step\s+\d+\s*/\s*\d+\s*$', '', last, flags=re.IGNORECASE).rstrip(".").strip()
        sigs.append({"levels": levels, "step_index": s_idx, "step_total": s_tot, "evidence": evidence})
    return sigs

json.loads(df.iloc[i]['Pathway_json'])
# Example quick check:
import random, json

idxs = random.sample(range(len(df)), 2000)

for i in idxs:
    raw = df.iloc[i]['Pathway']
    parsed_json = df.iloc[i]['Pathway_json']
    ref = extract_reference_signals(raw)

    if parsed_json is None or (isinstance(parsed_json, float) and pd.isna(parsed_json)):
        assert ref == [], f"Mismatch at row {i}"
    else:
        parsed = json.loads(parsed_json)
        assert ref == parsed, f"Mismatch at row {i}"


In [108]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 256690 entries, 0 to 256689
Data columns (total 29 columns):
 #   Column                              Non-Null Count   Dtype 
---  ------                              --------------   ----- 
 0   Entry                               256690 non-null  object
 1   name                                256690 non-null  object
 2   sequence                            256690 non-null  object
 3   function                            256690 non-null  object
 4   Organism                            256690 non-null  object
 5   Length                              256690 non-null  int64 
 6   Active site                         55837 non-null   object
 7   Catalytic activity                  132908 non-null  object
 8   Binding site                        115900 non-null  object
 9   Cofactor                            65251 non-null   object
 10  DNA binding                         4886 non-null    object
 11  EC number                           141

In [109]:
# save non null pathway rows
df[df['Pathway'].notna()][['Entry', 'Pathway', 'Pathway_json']].to_csv("temp/QA_Pathways_with_JSON.tsv", sep="\t", index=False)

### Subcellular location [CC]

In [111]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 256690 entries, 0 to 256689
Data columns (total 29 columns):
 #   Column                              Non-Null Count   Dtype 
---  ------                              --------------   ----- 
 0   Entry                               256690 non-null  object
 1   name                                256690 non-null  object
 2   sequence                            256690 non-null  object
 3   function                            256690 non-null  object
 4   Organism                            256690 non-null  object
 5   Length                              256690 non-null  int64 
 6   Active site                         55837 non-null   object
 7   Catalytic activity                  132908 non-null  object
 8   Binding site                        115900 non-null  object
 9   Cofactor                            65251 non-null   object
 10  DNA binding                         4886 non-null    object
 11  EC number                           141

In [110]:
df['Subcellular location [CC]'].unique()

array(['SUBCELLULAR LOCATION: Cell inner membrane {ECO:0000255|HAMAP-Rule:MF_01585}; Multi-pass membrane protein {ECO:0000255|HAMAP-Rule:MF_01585}.',
       'SUBCELLULAR LOCATION: Cytoplasm {ECO:0000255|HAMAP-Rule:MF_01109}.',
       'SUBCELLULAR LOCATION: Basolateral cell membrane; Multi-pass membrane protein.',
       ...,
       'SUBCELLULAR LOCATION: Golgi apparatus membrane {ECO:0000269|PubMed:10899003}; Single-pass type II membrane protein {ECO:0000269|PubMed:10899003}.',
       'SUBCELLULAR LOCATION: [Isoform 1]: Mitochondrion {ECO:0000269|PubMed:7599275}.; SUBCELLULAR LOCATION: [Isoform 2]: Mitochondrion {ECO:0000269|PubMed:7599275}. Nucleus inner membrane {ECO:0000269|PubMed:19602197, ECO:0000269|PubMed:7599275}; Peripheral membrane protein {ECO:0000269|PubMed:19602197, ECO:0000269|PubMed:7599275}; Nucleoplasmic side {ECO:0000269|PubMed:19602197, ECO:0000269|PubMed:7599275}. Note=Predominantly targeted to the nucleus. {ECO:0000269|PubMed:7599275}.',
       'SUBCELLULAR LOCATIO

In [112]:
import pandas as pd
import re, json

# ----- Patterns -----
SUBCELL_SPLIT = re.compile(r'SUBCELLULAR LOCATION:', flags=re.IGNORECASE)
EVID_BLOCK    = re.compile(r'\{([^}]*)\}')
NOTE_RX       = re.compile(r'Note=([^.{]+)')   # text after Note= up to '.' or '{'
ECO_RX        = re.compile(r'ECO:\d+')
PMID_RX       = re.compile(r'PubMed:\d+')

def parse_subcell_to_json(s: str):
    """
    Parse UniProt-style 'Subcellular location [CC]' into structured JSON.
    Schema:
      [
        {
          "location": str,            # main compartment
          "modifiers": [str],         # descriptors or subparts
          "note": str|null,           # note if present
          "evidence": [str]           # ECO/PMID/rule tokens
        },
        ...
      ]
    """
    if pd.isna(s) or not isinstance(s, str) or not s.strip():
        return None

    text = s.strip()
    # Split multiple SUBCELLULAR LOCATION: sentences
    segs = [seg.strip().rstrip('.') for seg in SUBCELL_SPLIT.split(text) if seg.strip()]
    out = []

    for seg in segs:
        # Split into statements by '.' then by ';'
        stmts = []
        for sent in seg.split('.'):
            sent = sent.strip()
            if not sent:
                continue
            stmts.extend([frag.strip() for frag in sent.split(';') if frag.strip()])

        for stmt in stmts:
            raw = stmt

            # Evidence in {...}
            evidence = []
            evid_matches = EVID_BLOCK.findall(raw)
            for evid_text in evid_matches:
                evidence.extend([tok.strip() for tok in re.split(r'[|,]', evid_text) if tok.strip()])
            # Remove evidence from raw
            raw = EVID_BLOCK.sub('', raw).strip()

            # Note=
            note = None
            m_note = NOTE_RX.search(raw)
            if m_note:
                note = m_note.group(1).strip()
                raw = NOTE_RX.sub('', raw).strip()

            # Also include inline ECO/PMID tokens if not captured
            for tok in ECO_RX.findall(stmt):
                if tok not in evidence:
                    evidence.append(tok)
            for tok in PMID_RX.findall(stmt):
                if tok not in evidence:
                    evidence.append(tok)

            # Split by comma into location + modifiers
            parts = [p.strip() for p in raw.split(',') if p.strip()]
            if not parts:
                continue
            location = parts[0]
            modifiers = parts[1:] if len(parts) > 1 else []

            out.append({
                "location": location,
                "modifiers": modifiers,
                "note": note,
                "evidence": evidence
            })

    return out if out else None


# ----- Apply to dataset -----
df['Subcellular_location_json'] = df['Subcellular location [CC]'].apply(
    lambda s: None if (obj := parse_subcell_to_json(s)) is None else json.dumps(obj, ensure_ascii=False)
)

# ----- Optional validation -----
# You can check a sample manually:
import random
for i in random.sample(range(len(df)), 20):
    print("\nRAW:", df.iloc[i]['Subcellular location [CC]'])
    print("PARSED:", df.iloc[i]['Subcellular_location_json'])

# ----- Save non null tsv -----
df[df['Subcellular location [CC]'].notna()][['Entry', 'Subcellular location [CC]', 'Subcellular_location_json']].to_csv("temp/QA_Subcellular_Locations_with_JSON.tsv", sep="\t", index=False)



RAW: nan
PARSED: None

RAW: nan
PARSED: None

RAW: nan
PARSED: None

RAW: SUBCELLULAR LOCATION: Cell membrane {ECO:0000255|HAMAP-Rule:MF_00386}; Peripheral membrane protein {ECO:0000255|HAMAP-Rule:MF_00386}; Cytoplasmic side {ECO:0000255|HAMAP-Rule:MF_00386}.
PARSED: [{"location": "Cell membrane", "modifiers": [], "note": null, "evidence": ["ECO:0000255", "HAMAP-Rule:MF_00386"]}, {"location": "Peripheral membrane protein", "modifiers": [], "note": null, "evidence": ["ECO:0000255", "HAMAP-Rule:MF_00386"]}, {"location": "Cytoplasmic side", "modifiers": [], "note": null, "evidence": ["ECO:0000255", "HAMAP-Rule:MF_00386"]}]

RAW: nan
PARSED: None

RAW: nan
PARSED: None

RAW: SUBCELLULAR LOCATION: Plastid, chloroplast thylakoid membrane {ECO:0000255|HAMAP-Rule:MF_01496}; Multi-pass membrane protein {ECO:0000255|HAMAP-Rule:MF_01496}.
PARSED: [{"location": "Plastid", "modifiers": ["chloroplast thylakoid membrane"], "note": null, "evidence": ["ECO:0000255", "HAMAP-Rule:MF_01496"]}, {"location

### Domain [FT]


In [113]:
import pandas as pd, re, json

# ---- Patterns ----
# Handle standard and qualified coordinates: "12..200", "<1..166", "10..>200", or single pos "42"
POS_RX        = re.compile(r'^\s*(?:<|>)?(\d+)(?:\.\.(?:<|>)?(\d+))?')
NOTE_RX       = re.compile(r'/note="([^"]+)"')
DESC_RX       = re.compile(r'/description="([^"]+)"')
DBX_ALL_RX    = re.compile(r'/db_xref="([^"]+)"')     # may appear multiple times
EVID_RX       = re.compile(r'/evidence="([^"]+)"')
ECO_INLINE_RX = re.compile(r'ECO:\d+')
PMID_INLINE_RX= re.compile(r'PubMed:\d+')

def parse_domain_ft_to_json(s: str):
    """
    Parse UniProt 'Domain [FT]' into a list of blocks with:
      start, end, note, description, db_xref(list), evidence(list).
    """
    if pd.isna(s) or not isinstance(s, str) or not s.strip():
        return None

    out = []
    for blk in s.strip().split('DOMAIN'):
        blk = blk.strip()
        if not blk:
            continue

        # positions
        mpos = POS_RX.match(blk)
        if not mpos:
            continue
        start = int(mpos.group(1))
        end = int(mpos.group(2)) if mpos.group(2) else start

        # note / description
        note = NOTE_RX.search(blk).group(1) if NOTE_RX.search(blk) else None
        description = DESC_RX.search(blk).group(1) if DESC_RX.search(blk) else None

        # db_xref: collect all
        db_xrefs = DBX_ALL_RX.findall(blk) or []

        # evidence: from /evidence= and inline ECO/PMID
        evidence = []
        me = EVID_RX.search(blk)
        if me:
            evidence.extend([t.strip() for t in me.group(1).split("|") if t.strip()])
        for tok in ECO_INLINE_RX.findall(blk):
            if tok not in evidence:
                evidence.append(tok)
        for tok in PMID_INLINE_RX.findall(blk):
            if tok not in evidence:
                evidence.append(tok)

        out.append({
            "start": start,
            "end": end,
            "note": note,
            "description": description,
            "db_xref": db_xrefs,
            "evidence": evidence
        })

    return out if out else None

# ---- Apply to your dataset ----
df['DomainFT_json'] = df['Domain [FT]'].apply(
    lambda s: None if (obj := parse_domain_ft_to_json(s)) is None else json.dumps(obj, ensure_ascii=False)
)

# ---- Optional: value-level validation on a large random sample ----
# (Independent re-extract using the same patterns to assert equality)
import random
def extract_reference_domain(raw: str):
    if pd.isna(raw) or not isinstance(raw, str) or not raw.strip():
        return []
    out = []
    for seg in raw.strip().split('DOMAIN'):
        seg = seg.strip()
        if not seg: continue
        mpos = POS_RX.match(seg)
        if not mpos: continue
        start = int(mpos.group(1))
        end = int(mpos.group(2)) if mpos.group(2) else start
        note = NOTE_RX.search(seg).group(1) if NOTE_RX.search(seg) else None
        desc = DESC_RX.search(seg).group(1) if DESC_RX.search(seg) else None
        dbxs = DBX_ALL_RX.findall(seg) or []
        evidence = []
        me = EVID_RX.search(seg)
        if me:
            evidence.extend([t.strip() for t in me.group(1).split("|") if t.strip()])
        for tok in ECO_INLINE_RX.findall(seg):
            if tok not in evidence: evidence.append(tok)
        for tok in PMID_INLINE_RX.findall(seg):
            if tok not in evidence: evidence.append(tok)
        out.append({
            "start": start, "end": end, "note": note,
            "description": desc, "db_xref": dbxs, "evidence": evidence
        })
    return out

# save to tsv
df[df['Domain [FT]'].notna()][['Entry', 'Domain [FT]', 'DomainFT_json']].to_csv("temp/QA_Domains_with_JSON.tsv", sep="\t", index=False)

### Motif, Topological domain

In [114]:
df['Motif'].unique()

array([nan,
       'MOTIF 14..22; /note="\'HIGH\' region"; /evidence="ECO:0000255|HAMAP-Rule:MF_00140"; MOTIF 217..221; /note="\'KMSKS\' region"; /evidence="ECO:0000255|HAMAP-Rule:MF_00140"',
       'MOTIF 90..102; /note="PRPP-binding"; /evidence="ECO:0000255|HAMAP-Rule:MF_01219"',
       ...,
       'MOTIF 401..405; /note="FXDXF motif"; /evidence="ECO:0000250|UniProtKB:Q8TEH3"; MOTIF 558..567; /note="Clathrin box"; /evidence="ECO:0000250|UniProtKB:Q8TEH3"',
       'MOTIF 347..349; /note="Microbody targeting signal"; /evidence="ECO:0000255"',
       'MOTIF 95..101; /note="Nuclear localization signal"'], dtype=object)

In [120]:
import pandas as pd, re, json

# ---- Patterns ----
# Optional isoform/accession prefix like "Q9NXH9-3:" then coordinate range.
POS_RX = re.compile(r'^\s*(?:[A-Za-z0-9]+(?:-[A-Za-z0-9]+)?:)?\s*(?:<|>)?(\d+)(?:\.\.(?:<|>)?(\d+))?')
NOTE_RX = re.compile(r'/note="([^"]+)"')
EVID_RX = re.compile(r'/evidence="([^"]+)"')
ECO_RX  = re.compile(r'ECO:\d+')
PMID_RX = re.compile(r'PubMed:\d+')

def parse_motif_to_json(s: str):
    """
    Parse UniProt-style MOTIF features into a list of dicts:
      {start:int, end:int, note:str|None, evidence:list[str]}
    """
    if pd.isna(s) or not isinstance(s, str) or not s.strip():
        return None

    out = []
    for blk in s.strip().split('MOTIF'):
        blk = blk.strip()
        if not blk:
            continue

        # coordinates
        mpos = POS_RX.match(blk)
        if not mpos:
            continue
        start = int(mpos.group(1))
        end   = int(mpos.group(2)) if mpos.group(2) else start

        # note
        note = NOTE_RX.search(blk).group(1) if NOTE_RX.search(blk) else None

        # evidence (block + inline tokens)
        evidence = []
        me = EVID_RX.search(blk)
        if me:
            evidence.extend([t.strip() for t in re.split(r'[|,]', me.group(1)) if t.strip()])
        for tok in ECO_RX.findall(blk):
            if tok not in evidence:
                evidence.append(tok)
        for tok in PMID_RX.findall(blk):
            if tok not in evidence:
                evidence.append(tok)

        out.append({
            "start": start,
            "end": end,
            "note": note,
            "evidence": evidence
        })

    return out if out else None

# ---- Apply to dataset ----
df['Motif_json'] = df['Motif'].apply(
    lambda s: None if (obj := parse_motif_to_json(s)) is None else json.dumps(obj, ensure_ascii=False)
)

# ---- Optional: value-level validation on a large sample ----
def extract_reference_motif(raw: str):
    if pd.isna(raw) or not isinstance(raw, str) or not raw.strip():
        return []
    out = []
    for seg in raw.strip().split('MOTIF'):
        seg = seg.strip()
        if not seg: continue
        mpos = POS_RX.match(seg)
        if not mpos: continue
        start = int(mpos.group(1))
        end = int(mpos.group(2)) if mpos.group(2) else start
        note = NOTE_RX.search(seg).group(1) if NOTE_RX.search(seg) else None
        evidence = []
        me = EVID_RX.search(seg)
        if me:
            evidence.extend([t.strip() for t in re.split(r'[|,]', me.group(1)) if t.strip()])
        for tok in ECO_RX.findall(seg):
            if tok not in evidence: evidence.append(tok)
        for tok in PMID_RX.findall(seg):
            if tok not in evidence: evidence.append(tok)
        out.append({"start": start, "end": end, "note": note, "evidence": evidence})
    return out

import random, json

idxs = random.sample(range(len(df)), 6000)

for i in idxs:
    raw = df.iloc[i]['Motif']
    ref = extract_reference_motif(raw)
    parsed_json = df.iloc[i]['Motif_json']

    if parsed_json is None or (isinstance(parsed_json, float) and pd.isna(parsed_json)):
        # For rows without Motif, both should be empty
        assert ref == [], f"Mismatch at row {i}"
    else:
        parsed = json.loads(parsed_json)
        assert ref == parsed, f"Mismatch at row {i}"


# ---- Save ----
df[df['Motif'].notna()][['Entry', 'Motif', 'Motif_json']].to_csv("temp/QA_Motifs_with_JSON.tsv", sep="\t", index=False)


In [115]:
df['Topological domain'].unique()

array(['TOPO_DOM 1..17; /note="Periplasmic"; /evidence="ECO:0000255|HAMAP-Rule:MF_01585"; TOPO_DOM 39..52; /note="Cytoplasmic"; /evidence="ECO:0000255|HAMAP-Rule:MF_01585"; TOPO_DOM 74..90; /note="Periplasmic"; /evidence="ECO:0000255|HAMAP-Rule:MF_01585"; TOPO_DOM 112..144; /note="Cytoplasmic"; /evidence="ECO:0000255|HAMAP-Rule:MF_01585"; TOPO_DOM 166; /note="Periplasmic"; /evidence="ECO:0000255|HAMAP-Rule:MF_01585"; TOPO_DOM 188..226; /note="Cytoplasmic"; /evidence="ECO:0000255|HAMAP-Rule:MF_01585"; TOPO_DOM 248..256; /note="Periplasmic"; /evidence="ECO:0000255|HAMAP-Rule:MF_01585"; TOPO_DOM 278..280; /note="Cytoplasmic"; /evidence="ECO:0000255|HAMAP-Rule:MF_01585"; TOPO_DOM 302..304; /note="Periplasmic"; /evidence="ECO:0000255|HAMAP-Rule:MF_01585"; TOPO_DOM 326..343; /note="Cytoplasmic"; /evidence="ECO:0000255|HAMAP-Rule:MF_01585"; TOPO_DOM 365..366; /note="Periplasmic"; /evidence="ECO:0000255|HAMAP-Rule:MF_01585"; TOPO_DOM 388..397; /note="Cytoplasmic"; /evidence="ECO:0000255|HAMAP-

In [121]:
import pandas as pd, re, json

# ---- Patterns ----
# Optional isoform/accession prefix like "Q9NXH9-3:"; supports open bounds < or >
POS_RX        = re.compile(r'^\s*(?:[A-Za-z0-9]+(?:-[A-Za-z0-9]+)?:)?\s*(?:<|>)?(\d+)(?:\.\.(?:<|>)?(\d+))?')
NOTE_RX       = re.compile(r'/note="([^"]+)"')
DESC_RX       = re.compile(r'/description="([^"]+)"')
DBX_ALL_RX    = re.compile(r'/db_xref="([^"]+)"')     # may occur multiple times
EVID_RX       = re.compile(r'/evidence="([^"]+)"')
ECO_RX        = re.compile(r'ECO:\d+')
PMID_RX       = re.compile(r'PubMed:\d+')

def parse_topodomain_to_json(s: str):
    """
    Parse UniProt-style 'Topological domain' (TOPO_DOM) into a list of blocks:
      start:int, end:int, note:str|None, description:str|None, db_xref:list[str], evidence:list[str]
    """
    if pd.isna(s) or not isinstance(s, str) or not s.strip():
        return None
    out = []
    for blk in s.strip().split('TOPO_DOM'):
        blk = blk.strip()
        if not blk:
            continue

        # Coordinates
        mpos = POS_RX.match(blk)
        if not mpos:
            continue
        start = int(mpos.group(1))
        end   = int(mpos.group(2)) if mpos.group(2) else start

        # Qualifiers
        note = NOTE_RX.search(blk).group(1) if NOTE_RX.search(blk) else None
        description = DESC_RX.search(blk).group(1) if DESC_RX.search(blk) else None
        db_xrefs = DBX_ALL_RX.findall(blk) or []

        # Evidence (from /evidence= and any inline ECO/PMID)
        evidence = []
        me = EVID_RX.search(blk)
        if me:
            evidence.extend([t.strip() for t in re.split(r'[|,]', me.group(1)) if t.strip()])
        for tok in ECO_RX.findall(blk):
            if tok not in evidence:
                evidence.append(tok)
        for tok in PMID_RX.findall(blk):
            if tok not in evidence:
                evidence.append(tok)

        out.append({
            "start": start,
            "end": end,
            "note": note,
            "description": description,
            "db_xref": db_xrefs,
            "evidence": evidence
        })
    return out if out else None

# ---- Apply to your dataset ----
df['Topological_domain_json'] = df['Topological domain'].apply(
    lambda s: None if (obj := parse_topodomain_to_json(s)) is None else json.dumps(obj, ensure_ascii=False)
)


df[df['Topological domain'].notna()][['Entry', 'Topological domain', 'Topological_domain_json']].to_csv("temp/QA_Topological_Domains_with_JSON.tsv", sep="\t", index=False)


In [122]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 256690 entries, 0 to 256689
Data columns (total 33 columns):
 #   Column                              Non-Null Count   Dtype 
---  ------                              --------------   ----- 
 0   Entry                               256690 non-null  object
 1   name                                256690 non-null  object
 2   sequence                            256690 non-null  object
 3   function                            256690 non-null  object
 4   Organism                            256690 non-null  object
 5   Length                              256690 non-null  int64 
 6   Active site                         55837 non-null   object
 7   Catalytic activity                  132908 non-null  object
 8   Binding site                        115900 non-null  object
 9   Cofactor                            65251 non-null   object
 10  DNA binding                         4886 non-null    object
 11  EC number                           141

# Gene Ontology (biological process), Gene Ontology (cellular component), Gene Ontology (molecular function)

In [116]:
df['Gene Ontology (biological process)'].unique()

array([nan,
       'arginine biosynthetic process via ornithine [GO:0042450]; citrulline biosynthetic process [GO:0019240]; L-arginine biosynthetic process [GO:0006526]',
       'regulation of intracellular pH [GO:0051453]; sodium ion import across plasma membrane [GO:0098719]',
       ...,
       'compound eye development [GO:0048749]; compound eye morphogenesis [GO:0001745]; cuticle pattern formation [GO:0035017]; dorsal/ventral pattern formation, imaginal disc [GO:0007450]; female germ-line stem cell population maintenance [GO:0036099]; follicle cell of egg chamber development [GO:0030707]; germarium-derived egg chamber formation [GO:0007293]; imaginal disc-derived leg morphogenesis [GO:0007480]; imaginal disc-derived leg segmentation [GO:0036011]; imaginal disc-derived wing margin morphogenesis [GO:0008587]; imaginal disc-derived wing morphogenesis [GO:0007476]; negative regulation of Notch signaling pathway [GO:0045746]; positive regulation of Notch signaling pathway [GO:0045747];

In [123]:
import pandas as pd, re, json

# ---- Patterns ----
ITEM_SPLIT = re.compile(r'\s*;\s*')                 # split items
GO_ITEM_RX = re.compile(r'^(.*?)\s*\[(GO:\d{7})\]\s*$')
GO_ID_ANY  = re.compile(r'(GO:\d{7})')

def parse_go_bp_to_json(s: str):
    """
    Parse 'Gene Ontology (biological process)' entries into list of:
      {go_id:str, term:str, aspect:'BP', evidence:list[str]}.
    """
    if pd.isna(s) or not isinstance(s, str) or not s.strip():
        return None
    text = s.strip()
    items = [p for p in ITEM_SPLIT.split(text) if p.strip()]
    out = []
    for it in items:
        it = it.strip()
        m = GO_ITEM_RX.match(it)
        if m:
            term = m.group(1).strip()
            go_id = m.group(2)
        else:
            # Fallback: find GO id anywhere, take the text before it as the term
            mid = GO_ID_ANY.search(it)
            if not mid:
                continue
            go_id = mid.group(1)
            bpos = it.find('[' + go_id + ']')
            term = it[:bpos].strip() if bpos != -1 else it[:mid.start()].strip()

        if term.endswith(','):
            term = term[:-1].rstrip()

        out.append({"go_id": go_id, "term": term, "aspect": "BP", "evidence": []})
    return out if out else None

# ---- Apply to dataset ----
col_bp = 'Gene Ontology (biological process)'
df['GO_BP_json'] = df[col_bp].apply(
    lambda s: None if (obj := parse_go_bp_to_json(s)) is None else json.dumps(obj, ensure_ascii=False)
)


# ---- Save ----
df[df[col_bp].notna()][['Entry', col_bp, 'GO_BP_json']].to_csv("temp/QA_GO_BP_with_JSON.tsv", sep="\t", index=False)

In [117]:
df['Gene Ontology (cellular component)'].unique()

array(['plasma membrane [GO:0005886]', 'cytoplasm [GO:0005737]',
       'basolateral plasma membrane [GO:0016323]', ...,
       'cytosol [GO:0005829]; early endosome [GO:0005769]; endocytic vesicle [GO:0030139]; recycling endosome [GO:0055037]',
       'cytoplasm [GO:0005737]; Golgi medial cisterna [GO:0005797]; Golgi membrane [GO:0000139]; Golgi stack [GO:0005795]',
       'mitochondrion [GO:0005739]; nuclear envelope [GO:0005635]; nuclear inner membrane [GO:0005637]; nucleus [GO:0005634]'],
      dtype=object)

In [124]:
import pandas as pd, re, json

# ---- Patterns ----
ITEM_SPLIT = re.compile(r'\s*;\s*')
GO_ITEM_RX = re.compile(r'^(.*?)\s*\[(GO:\d{7})\]\s*$')
GO_ID_ANY  = re.compile(r'(GO:\d{7})')

def parse_go_cc_to_json(s: str):
    """
    Parse 'Gene Ontology (cellular component)' items into:
      {go_id:str, term:str, aspect:'CC', evidence:list[str]}
    """
    if pd.isna(s) or not isinstance(s, str) or not s.strip():
        return None
    text = s.strip()
    items = [p for p in ITEM_SPLIT.split(text) if p.strip()]
    out = []
    for it in items:
        it = it.strip()
        m = GO_ITEM_RX.match(it)
        if m:
            term = m.group(1).strip()
            go_id = m.group(2)
        else:
            # Fallback: find GO id anywhere, use text before it as term
            mid = GO_ID_ANY.search(it)
            if not mid:
                continue
            go_id = mid.group(1)
            bpos = it.find('[' + go_id + ']')
            term = it[:bpos].strip() if bpos != -1 else it[:mid.start()].strip()
        if term.endswith(','):
            term = term[:-1].rstrip()
        out.append({"go_id": go_id, "term": term, "aspect": "CC", "evidence": []})
    return out if out else None

# ---- Apply to dataset ----
col_cc = 'Gene Ontology (cellular component)'
df['GO_CC_json'] = df[col_cc].apply(
    lambda s: None if (obj := parse_go_cc_to_json(s)) is None else json.dumps(obj, ensure_ascii=False)
)

# ---- Optional: value-level validation on a large sample ----
def extract_reference_go_cc(raw: str):
    if pd.isna(raw) or not isinstance(raw, str) or not raw.strip():
        return []
    items = [p for p in ITEM_SPLIT.split(raw.strip()) if p.strip()]
    sigs = []
    for it in items:
        it = it.strip()
        m = GO_ITEM_RX.match(it)
        if m:
            term = m.group(1).strip(); go_id = m.group(2)
        else:
            mid = GO_ID_ANY.search(it)
            if not mid: continue
            go_id = mid.group(1)
            bpos = it.find('[' + go_id + ']')
            term = it[:bpos].strip() if bpos != -1 else it[:mid.start()].strip()
        if term.endswith(','): term = term[:-1].rstrip()
        sigs.append({"go_id": go_id, "term": term, "aspect": "CC", "evidence": []})
    return sigs

# ---- Save ----
df[df[col_cc].notna()][['Entry', col_cc, 'GO_CC_json']].to_csv("temp/QA_GO_CC_with_JSON.tsv", sep="\t", index=False)

In [118]:
df['Gene Ontology (molecular function)'].unique()

array(['lysophospholipid:sodium symporter activity [GO:0051978]',
       'amino acid binding [GO:0016597]; ornithine carbamoyltransferase activity [GO:0004585]',
       'potassium:proton antiporter activity [GO:0015386]; sodium:proton antiporter activity [GO:0015385]',
       ...,
       'DNA binding [GO:0003677]; guanyl-nucleotide exchange factor activity [GO:0005085]; small GTPase binding [GO:0031267]; zinc ion binding [GO:0008270]',
       'DNA-binding transcription activator activity, RNA polymerase II-specific [GO:0001228]; RNA polymerase II cis-regulatory region sequence-specific DNA binding [GO:0000978]; sequence-specific DNA binding [GO:0043565]; sequence-specific double-stranded DNA binding [GO:1990837]',
       'tRNA (guanine(26)-N2)-dimethyltransferase activity [GO:0160104]; tRNA (guanine(26)-N2/guanine(27)-N2)-dimethyltransferase activity [GO:0160103]; tRNA binding [GO:0000049]'],
      dtype=object)

In [125]:
import pandas as pd, re, json

# ---- Patterns ----
ITEM_SPLIT = re.compile(r'\s*;\s*')
GO_ITEM_RX = re.compile(r'^(.*?)\s*\[(GO:\d{7})\]\s*$')
GO_ID_ANY  = re.compile(r'(GO:\d{7})')

def parse_go_mf_to_json(s: str):
    """
    Parse 'Gene Ontology (molecular function)' items into:
      {go_id:str, term:str, aspect:'MF', evidence:list[str]}
    """
    if pd.isna(s) or not isinstance(s, str) or not s.strip():
        return None
    items = [p for p in ITEM_SPLIT.split(s.strip()) if p.strip()]
    out = []
    for it in items:
        it = it.strip()
        m = GO_ITEM_RX.match(it)
        if m:
            term = m.group(1).strip()
            go_id = m.group(2)
        else:
            # Fallback: find GO id anywhere, use text before it as term
            mid = GO_ID_ANY.search(it)
            if not mid:
                continue
            go_id = mid.group(1)
            bpos = it.find('[' + go_id + ']')
            term = it[:bpos].strip() if bpos != -1 else it[:mid.start()].strip()
        if term.endswith(','):
            term = term[:-1].rstrip()
        out.append({"go_id": go_id, "term": term, "aspect": "MF", "evidence": []})
    return out if out else None

# ---- Apply to dataset ----
col_mf = 'Gene Ontology (molecular function)'
df['GO_MF_json'] = df[col_mf].apply(
    lambda s: None if (obj := parse_go_mf_to_json(s)) is None else json.dumps(obj, ensure_ascii=False)
)

# ---- Optional: value-level validation on a large sample ----
def extract_reference_go_mf(raw: str):
    if pd.isna(raw) or not isinstance(raw, str) or not raw.strip():
        return []
    items = [p for p in ITEM_SPLIT.split(raw.strip()) if p.strip()]
    sigs = []
    for it in items:
        it = it.strip()
        m = GO_ITEM_RX.match(it)
        if m:
            term = m.group(1).strip(); go_id = m.group(2)
        else:
            mid = GO_ID_ANY.search(it)
            if not mid: continue
            go_id = mid.group(1)
            bpos = it.find('[' + go_id + ']')
            term = it[:bpos].strip() if bpos != -1 else it[:mid.start()].strip()
        if term.endswith(','): term = term[:-1].rstrip()
        sigs.append({"go_id": go_id, "term": term, "aspect": "MF", "evidence": []})
    return sigs


# ---- Save ----
df[df[col_mf].notna()][['Entry', col_mf, 'GO_MF_json']].to_csv("temp/QA_GO_MF_with_JSON.tsv", sep="\t", index=False)

In [126]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 256690 entries, 0 to 256689
Data columns (total 36 columns):
 #   Column                              Non-Null Count   Dtype 
---  ------                              --------------   ----- 
 0   Entry                               256690 non-null  object
 1   name                                256690 non-null  object
 2   sequence                            256690 non-null  object
 3   function                            256690 non-null  object
 4   Organism                            256690 non-null  object
 5   Length                              256690 non-null  int64 
 6   Active site                         55837 non-null   object
 7   Catalytic activity                  132908 non-null  object
 8   Binding site                        115900 non-null  object
 9   Cofactor                            65251 non-null   object
 10  DNA binding                         4886 non-null    object
 11  EC number                           141

In [129]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 256690 entries, 0 to 256689
Data columns (total 36 columns):
 #   Column                              Non-Null Count   Dtype 
---  ------                              --------------   ----- 
 0   Entry                               256690 non-null  object
 1   name                                256690 non-null  object
 2   sequence                            256690 non-null  object
 3   function                            256690 non-null  object
 4   Organism                            256690 non-null  object
 5   Length                              256690 non-null  int64 
 6   Active site                         55837 non-null   object
 7   Catalytic activity                  132908 non-null  object
 8   Binding site                        115900 non-null  object
 9   Cofactor                            65251 non-null   object
 10  DNA binding                         4886 non-null    object
 11  EC number                           141

In [130]:
df2 = pd.read_csv("main/Main_Data_Parsed.csv")

uniprot_seq_df = pd.read_csv("main/seq/UniProt_Sequence.tsv", sep="\t")

# join with main df on Entry, change the new column name to 'sequence_UniProt'
uniprot_seq_df.rename(columns={'Entry': 'Entry', 'Sequence': 'sequence_UniProt'}, inplace=True)
df2 = df2.merge(uniprot_seq_df[['Entry', 'sequence_UniProt']], on='Entry', how='left')


In [131]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 256690 entries, 0 to 256689
Data columns (total 37 columns):
 #   Column                              Non-Null Count   Dtype 
---  ------                              --------------   ----- 
 0   Entry                               256690 non-null  object
 1   name                                256690 non-null  object
 2   sequence                            256690 non-null  object
 3   function                            256690 non-null  object
 4   Organism                            256690 non-null  object
 5   Length                              256690 non-null  int64 
 6   Active site                         55837 non-null   object
 7   Catalytic activity                  132908 non-null  object
 8   Binding site                        115900 non-null  object
 9   Cofactor                            65251 non-null   object
 10  DNA binding                         4886 non-null    object
 11  EC number                           141

In [132]:
# sequence                            256690 non-null  object
# sequence_UniProt                   256681 non-null  object

# check how many sequences are different
df2['sequence'].fillna('', inplace=True)
df2['sequence_UniProt'].fillna('', inplace=True)
df2['seq_match'] = df2['sequence'] == df2['sequence_UniProt']   

/var/folders/7g/nsys8v9n73ddsfkrmy2zz58w0000gn/T/ipykernel_2164/3621565579.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df2['sequence'].fillna('', inplace=True)
/var/folders/7g/nsys8v9n73ddsfkrmy2zz58w0000gn/T/ipykernel_2164/3621565579.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves

In [133]:
df2['seq_match'].value_counts()

seq_match
True     256295
False       395
Name: count, dtype: int64

In [134]:
# show the rows where the sequences are different
df2[df2['seq_match'] == False][['Entry', 'sequence', 'sequence_UniProt']]

,Entry,sequence,sequence_UniProt
18,A0A1W2PPG7,MSSKVAINSDIGQALWAVEQLQMEAGIDQVKMAADLLKFCTEQAKN...,MSSKVAINSDIGQALWAVEQLQMEAGIDQVKVRVGASAGGGKRWEH...
576,Q2U0K2,MWWPRATADRLQTCTFWFLWLFTWDDEIDQSTSDLFIHIHKANDFR...,MSQSVVVNPRPALKAKLTKEPFSIPNLKPYYKAWPTAVNPDYPGLK...
627,Q4IBL8,MREIVTLQLDQESYFTYSSDEKSLIDHNVHWRAGLGADGSETFLPR...,MREIVTLQLGQLSNYTATHFWNAQESYFTYSSDEKSLIDHNVHWRA...
832,Q8NGJ2,MPSASAMIIFNLSSYNPGPFILVGIPGLEQFHVWIGIPFCIIYIVA...,MIIFNLSSYNPGPFILVGIPGLEQFHVWIGIPFCIIYIVAVVGNCI...
844,Q99490,MSRGAGALQRRTTTYLISLTLVKLESVPPPPPSPSAAAVGAPGARG...,MSRGAGALQRRTTTYLISLTLVKLESVPPPPPSPSAAAAGAAGARG...
...,...,...,...
255845,Q5K8L4,MASKQHAHILSLARSMIPPLHPKLHKGQAGRIGVLGGSGDYSGAPY...,MASKQHAHILSLARSMIPPLHPKLHKGQAGRIGVLGGSGDYSGAPY...
256322,O13861,METLQVEVFFLFDFTSIMAGFKVPSWITYKSFWIAVSSSVTTACVI...,MAGFKVPSWITYKSFWIAVSSSVTTACVILGTLEFRKHRSIRRLQS...
256323,P34331,MNRLPNIAKPPQKSNQRKEKAPPEVPALIADKDRGTYYEKGRFLGK...,MNRLPNIAKPPQKSNQRKEKAPPEVPALIADKDRGTYYEKGRFLGK...
256513,Q10337,MEKIKLLNVKTPNHYTIIFKVVAYYSALQPNQNELRKKELFIKNDG...,MLAAESSNKELFIKNDGKALSFPYDWKLATHVICDDFSSPNVQEGS...


In [136]:
# values where sequence is different, why is it different?
# 3 col tsv
df2[df2['seq_match'] == False][['Entry', 'sequence', 'sequence_UniProt']].to_csv("temp/QA_Sequence_Mismatches.tsv", sep="\t", index=False)

In [137]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 256690 entries, 0 to 256689
Data columns (total 38 columns):
 #   Column                              Non-Null Count   Dtype 
---  ------                              --------------   ----- 
 0   Entry                               256690 non-null  object
 1   name                                256690 non-null  object
 2   sequence                            256690 non-null  object
 3   function                            256690 non-null  object
 4   Organism                            256690 non-null  object
 5   Length                              256690 non-null  int64 
 6   Active site                         55837 non-null   object
 7   Catalytic activity                  132908 non-null  object
 8   Binding site                        115900 non-null  object
 9   Cofactor                            65251 non-null   object
 10  DNA binding                         4886 non-null    object
 11  EC number                           141

In [138]:
df2.to_csv("main/Main_Data_Parsed.csv", index=False)

### Sequence fixing and final df

In [2]:
import pandas as pd

In [16]:
final_df = pd.read_csv("main/Main_Data_Parsed.csv")

In [17]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 256690 entries, 0 to 256689
Data columns (total 38 columns):
 #   Column                              Non-Null Count   Dtype 
---  ------                              --------------   ----- 
 0   Entry                               256690 non-null  object
 1   name                                256690 non-null  object
 2   sequence                            256690 non-null  object
 3   function                            256690 non-null  object
 4   Organism                            256690 non-null  object
 5   Length                              256690 non-null  int64 
 6   Active site                         55837 non-null   object
 7   Catalytic activity                  132908 non-null  object
 8   Binding site                        115900 non-null  object
 9   Cofactor                            65251 non-null   object
 10  DNA binding                         4886 non-null    object
 11  EC number                           141

In [18]:
mask = (final_df['sequence'] != final_df['sequence_UniProt']) & final_df['sequence_UniProt'].notna() & (final_df['sequence_UniProt'] != '')

final_df.loc[mask, 'sequence'] = final_df.loc[mask, 'sequence_UniProt']

replaced_count = mask.sum()
print("Total replaced:", replaced_count)

Total replaced: 386


In [19]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 256690 entries, 0 to 256689
Data columns (total 38 columns):
 #   Column                              Non-Null Count   Dtype 
---  ------                              --------------   ----- 
 0   Entry                               256690 non-null  object
 1   name                                256690 non-null  object
 2   sequence                            256690 non-null  object
 3   function                            256690 non-null  object
 4   Organism                            256690 non-null  object
 5   Length                              256690 non-null  int64 
 6   Active site                         55837 non-null   object
 7   Catalytic activity                  132908 non-null  object
 8   Binding site                        115900 non-null  object
 9   Cofactor                            65251 non-null   object
 10  DNA binding                         4886 non-null    object
 11  EC number                           141

In [20]:
final_df.drop(columns=['sequence_UniProt', 'seq_match'], inplace=True)

In [21]:
final_df.drop(columns=['Active site', 'Binding site', 'Catalytic activity', 'Cofactor', 'DNA binding', 'Pathway', 'Subcellular location [CC]', 'Domain [FT]', 'Gene Ontology (biological process)', 'Gene Ontology (cellular component)',  'Gene Ontology (molecular function)',  'Motif', 'Topological domain'], inplace=True)

In [22]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 256690 entries, 0 to 256689
Data columns (total 23 columns):
 #   Column                     Non-Null Count   Dtype 
---  ------                     --------------   ----- 
 0   Entry                      256690 non-null  object
 1   name                       256690 non-null  object
 2   sequence                   256690 non-null  object
 3   function                   256690 non-null  object
 4   Organism                   256690 non-null  object
 5   Length                     256690 non-null  int64 
 6   EC number                  141536 non-null  object
 7   Reactome                   18470 non-null   object
 8   UniPathway                 66801 non-null   object
 9   length                     256690 non-null  int64 
 10  Active_site_json           55837 non-null   object
 11  Binding_site_json          115897 non-null  object
 12  Catalytic_Activity_json    132908 non-null  object
 13  Cofactor_json              65245 non-null   

In [24]:
final_df['Length'] = final_df['sequence'].str.len()


In [26]:
final_df.drop(columns=['length'], inplace=True) 

In [27]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 256690 entries, 0 to 256689
Data columns (total 22 columns):
 #   Column                     Non-Null Count   Dtype 
---  ------                     --------------   ----- 
 0   Entry                      256690 non-null  object
 1   name                       256690 non-null  object
 2   sequence                   256690 non-null  object
 3   function                   256690 non-null  object
 4   Organism                   256690 non-null  object
 5   Length                     256690 non-null  int64 
 6   EC number                  141536 non-null  object
 7   Reactome                   18470 non-null   object
 8   UniPathway                 66801 non-null   object
 9   Active_site_json           55837 non-null   object
 10  Binding_site_json          115897 non-null  object
 11  Catalytic_Activity_json    132908 non-null  object
 12  Cofactor_json              65245 non-null   object
 13  DNA_binding_json           4886 non-null    

In [28]:
# save the df
final_df.to_csv("main/Main_data_final_cols.csv", index=False)